# Knowledge Graphs Tutorial: A Complete Guide to Neo4j and Cypher

## 🎯 Learning Objectives
By the end of this tutorial, you will:
- Understand what Knowledge Graphs are and why they're valuable
- Learn the fundamentals of graph database concepts
- Master basic to advanced Cypher query language
- Build and query a movie knowledge graph using real data
- Apply graph algorithms for data analysis

## 📋 Prerequisites
- Basic understanding of databases
- Python programming fundamentals
- No prior Neo4j or graph database experience required!

---

## 1. What are Knowledge Graphs?

### 🔍 Definition
A **Knowledge Graph** is a network of real-world entities (people, places, things, concepts) and their relationships, stored in a graph database format.

### 🏗️ Core Components
1. **Nodes (Vertices)**: Represent entities (e.g., Person, Movie, Company)
2. **Relationships (Edges)**: Connect nodes and represent how entities relate
3. **Properties**: Attributes that describe nodes and relationships
4. **Labels**: Categories that group similar nodes

### 📊 Graph vs. Traditional Databases

| Aspect | Relational Database | Knowledge Graph |
|--------|-------------------|----------------|
| **Structure** | Tables with rows/columns | Nodes connected by relationships |
| **Relationships** | Foreign keys, JOINs | Direct connections |
| **Schema** | Fixed schema | Flexible, schema-optional |
| **Queries** | SQL | Cypher, SPARQL |
| **Best For** | Structured data, transactions | Connected data, pattern detection |

### 🌟 Why Use Knowledge Graphs?
- **Intuitive**: Mirrors how humans think about relationships
- **Flexible**: Easy to add new entity types and relationships
- **Performance**: Fast traversal of connected data
- **Insights**: Reveals hidden patterns and connections
- **Integration**: Combines data from multiple sources naturally

### 🚀 Real-World Applications
- **Recommendation Systems**: Netflix, Amazon product recommendations
- **Fraud Detection**: Banking, insurance claim analysis
- **Social Networks**: Facebook's social graph, LinkedIn connections
- **Knowledge Management**: Google's Knowledge Graph, Wikipedia
- **Drug Discovery**: Understanding protein interactions
- **Supply Chain**: Tracking dependencies and risks

## 2. Introduction to Neo4j

### 🗄️ What is Neo4j?
Neo4j is the world's leading **native graph database**, designed from the ground up to store and query connected data efficiently.

### 🏷️ Neo4j Graph Model
- **Labeled Property Graph**: Nodes have labels, relationships have types, both can have properties
- **ACID Compliance**: Ensures data integrity with transactions
- **Index-Free Adjacency**: Relationships are stored as pointers, enabling fast traversals

### 📝 Cypher Query Language
**Cypher** is Neo4j's declarative query language, designed to be:
- **Visual**: Syntax resembles graph drawings
- **Expressive**: Handles complex graph patterns
- **Readable**: Close to natural language

#### Basic Cypher Syntax Patterns:
```cypher
// Node pattern
(n)           // Any node
(n:Person)    // Node with label 'Person'
(n:Person {name: "Alice"})  // Node with label and property

// Relationship pattern  
-[:KNOWS]->   // Directed relationship
-[:KNOWS]-    // Undirected relationship
-[r:KNOWS]->  // Relationship with variable

// Complete pattern
(alice:Person)-[:KNOWS]->(bob:Person)
```

## 3. Environment Setup

Let's set up our development environment and connect to Neo4j.

In [ ]:
# Import required libraries
from dotenv import load_dotenv
import os
import pandas as pd
from langchain_community.graphs import Neo4jGraph

# Warning control
import warnings
warnings.filterwarnings("ignore")

print("✅ Libraries imported successfully!")

In [ ]:
# Load environment variables
load_dotenv('.env', override=True)

# Neo4j connection parameters
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE')

# Initialize Neo4j connection
kg = Neo4jGraph(
    url=NEO4J_URI, 
    username=NEO4J_USERNAME, 
    password=NEO4J_PASSWORD, 
    database=NEO4J_DATABASE
)

print("🔌 Connected to Neo4j database successfully!")
print(f"📍 Database URI: {NEO4J_URI}")

### 🔍 Neo4j Connection Health Check\n\nBefore we start, let's verify that your Neo4j Aura instance is running and accessible:"

In [ ]:
def check_neo4j_connection():\n    \"\"\"\n    Comprehensive Neo4j connection health check\n    \"\"\"\n    print(\"🔍 Neo4j Connection Health Check\")\n    print(\"=\" * 40)\n    \n    # Step 1: Check environment variables\n    print(\"\\n1️⃣ Environment Variables:\")\n    required_vars = ['NEO4J_URI', 'NEO4J_USERNAME', 'NEO4J_PASSWORD', 'NEO4J_DATABASE']\n    env_status = {}\n    \n    for var in required_vars:\n        value = os.getenv(var)\n        if value:\n            # Mask sensitive information\n            if 'PASSWORD' in var:\n                display_value = '*' * len(value)\n            elif 'URI' in var:\n                display_value = value\n            else:\n                display_value = value\n            print(f\"   ✅ {var}: {display_value}\")\n            env_status[var] = True\n        else:\n            print(f\"   ❌ {var}: Not set\")\n            env_status[var] = False\n    \n    if not all(env_status.values()):\n        print(\"\\n   ⚠️ Missing environment variables! Please check your .env file.\")\n        return False\n    \n    # Step 2: Test basic connectivity\n    print(\"\\n2️⃣ Testing Connection:\")\n    try:\n        # Simple connectivity test\n        test_kg = Neo4jGraph(\n            url=NEO4J_URI, \n            username=NEO4J_USERNAME, \n            password=NEO4J_PASSWORD, \n            database=NEO4J_DATABASE\n        )\n        \n        # Try a simple query\n        result = test_kg.query(\"RETURN 'Connection successful' AS status, datetime() AS timestamp\")\n        if result:\n            print(f\"   ✅ Connection successful!\")\n            print(f\"   📅 Server timestamp: {result[0].get('timestamp', 'N/A')}\")\n            return True\n        else:\n            print(\"   ❌ Query returned no results\")\n            return False\n            \n    except Exception as e:\n        print(f\"   ❌ Connection failed: {str(e)}\")\n        \n        # Provide specific guidance based on error type\n        error_msg = str(e).lower()\n        print(\"\\n🔧 Troubleshooting Guide:\")\n        \n        if 'authentication' in error_msg or 'credentials' in error_msg:\n            print(\"   • Check your username and password in .env file\")\n            print(\"   • Verify credentials in Neo4j Aura Console\")\n            \n        elif 'connection refused' in error_msg or 'timeout' in error_msg:\n            print(\"   • Your Neo4j Aura instance may be paused/stopped\")\n            print(\"   • Go to https://console.neo4j.io to resume your instance\")\n            print(\"   • Wait 60 seconds after resuming before retrying\")\n            \n        elif 'ssl' in error_msg or 'certificate' in error_msg:\n            print(\"   • SSL/TLS connection issue\")\n            print(\"   • Ensure your URI starts with 'neo4j+s://'\")\n            \n        elif 'database' in error_msg:\n            print(\"   • Database name may be incorrect\")\n            print(\"   • Check NEO4J_DATABASE in your .env file\")\n            \n        else:\n            print(f\"   • General connection error: {str(e)[:100]}...\")\n            \n        print(\"\\n📋 Quick Fix Steps:\")\n        print(\"   1. Go to https://console.neo4j.io\")\n        print(\"   2. Find your Aura instance\")\n        print(\"   3. Click 'Resume' if status shows 'Paused'\")\n        print(\"   4. Wait 60 seconds for instance to start\")\n        print(\"   5. Re-run this cell\")\n        \n        return False\n\n# Run the connection check\nconnection_ok = check_neo4j_connection()\n\nif connection_ok:\n    print(\"\\n🎉 Ready to proceed with the tutorial!\")\nelse:\n    print(\"\\n⏸️ Please fix the connection issues above before continuing.\")"

In [ ]:
# Only proceed with connection if health check passed\nif 'connection_ok' not in locals() or not connection_ok:\n    print(\"❌ Skipping connection setup - please fix connection issues first\")\nelse:\n    # Initialize Neo4j connection\n    kg = Neo4jGraph(\n        url=NEO4J_URI, \n        username=NEO4J_USERNAME, \n        password=NEO4J_PASSWORD, \n        database=NEO4J_DATABASE\n    )\n    \n    print(\"🔌 Connected to Neo4j database successfully!\")\n    print(f\"📍 Database URI: {NEO4J_URI}\")\n    \n    # Get basic instance information\n    try:\n        result = kg.query(\"CALL dbms.components() YIELD name, versions, edition RETURN name, versions[0] AS version, edition\")\n        if result:\n            component = result[0]\n            print(f\"📊 Neo4j {component.get('edition', 'Unknown')} Edition - Version {component.get('version', 'Unknown')}\")\n    except:\n        print(\"📊 Instance information not available\")

### 🔄 Retry Connection Helper\n\nIf your connection failed, you can use this cell to retry after fixing the issue:"

In [ ]:
# Retry connection helper - run this after fixing issues\ndef retry_connection():\n    print(\"🔄 Retrying Neo4j connection...\")\n    global kg, connection_ok\n    \n    # Re-run the health check\n    connection_ok = check_neo4j_connection()\n    \n    if connection_ok:\n        # Re-establish the connection\n        kg = Neo4jGraph(\n            url=NEO4J_URI, \n            username=NEO4J_USERNAME, \n            password=NEO4J_PASSWORD, \n            database=NEO4J_DATABASE\n        )\n        print(\"\\n✅ Connection retry successful!\")\n        return True\n    else:\n        print(\"\\n❌ Connection retry failed. Please check the issues above.\")\n        return False\n\n# Uncomment the line below to retry connection:\n# retry_connection()"

### ⚠️ Important Note about Neo4j Aura\n\n**Neo4j Aura Free instances automatically pause after 3 days of inactivity.** This is normal behavior and helps manage resources.\n\n#### If your instance is paused:\n1. Go to [Neo4j Aura Console](https://console.neo4j.io)\n2. Find your instance in the dashboard\n3. Click the **\"Resume\"** button\n4. Wait **60 seconds** for the instance to fully start\n5. Re-run the connection health check above\n\n#### Common connection issues:\n- **\"Connection refused\"**: Instance is paused - follow steps above\n- **\"Authentication failed\"**: Check credentials in `.env` file\n- **\"SSL/TLS errors\"**: Ensure URI starts with `neo4j+s://`\n- **\"Database not found\"**: Verify database name (usually \"neo4j\")\n\n---"

## 4. Cypher Fundamentals

### 📚 Core Cypher Clauses

| Clause | Purpose | Example |
|--------|---------|--------|
| `MATCH` | Find patterns in the graph | `MATCH (n:Person)` |
| `CREATE` | Create nodes and relationships | `CREATE (n:Person {name: "Alice"})` |
| `MERGE` | Create if doesn't exist, match if exists | `MERGE (n:Person {name: "Alice"})` |
| `SET` | Update properties | `SET n.age = 30` |
| `DELETE` | Remove nodes/relationships | `DELETE r` |
| `RETURN` | Specify what to output | `RETURN n.name, n.age` |
| `WHERE` | Filter results | `WHERE n.age > 25` |
| `ORDER BY` | Sort results | `ORDER BY n.name ASC` |
| `LIMIT` | Restrict number of results | `LIMIT 10` |

### 🔧 Essential Functions
- `count()`: Count nodes/relationships
- `collect()`: Aggregate values into lists
- `size()`: Get collection size
- `exists()`: Check if property/pattern exists
- `toLower()`, `toUpper()`: String manipulation

Let's start with basic queries to understand our current graph state:

In [ ]:
# Check current graph state
print("🔍 Current Graph Overview:")
print("-" * 50)

# Count all nodes
result = kg.query("""
MATCH (n) 
RETURN count(n) AS totalNodes
""")
print(f"📊 Total nodes: {result[0]['totalNodes']}")

# Count nodes by label
result = kg.query("""
MATCH (n)
RETURN labels(n)[0] AS nodeLabel, count(n) AS count
ORDER BY count DESC
""")
print("\n📋 Nodes by type:")
for row in result:
    if row['nodeLabel']:  # Skip nodes without labels
        print(f"   {row['nodeLabel']}: {row['count']}")

# Count all relationships
result = kg.query("""
MATCH ()-[r]->()
RETURN count(r) AS totalRelationships
""")
print(f"\n🔗 Total relationships: {result[0]['totalRelationships']}")

In [ ]:
# Safety check before proceeding with queries\ndef check_kg_available():\n    if 'kg' not in globals():\n        print(\"❌ Neo4j connection not established!\")\n        print(\"   Please run the connection health check above first.\")\n        return False\n    \n    if not globals().get('connection_ok', False):\n        print(\"❌ Neo4j connection health check failed!\")\n        print(\"   Please fix connection issues before proceeding.\")\n        return False\n    \n    return True\n\n# This will be used throughout the notebook to ensure connection is available\nif check_kg_available():\n    print(\"✅ Neo4j connection verified - ready for queries!\")\nelse:\n    print(\"⏸️ Please establish connection first before running subsequent cells.\")"

## 5. Loading Data from CSV Files

### 📁 Data Structure
We'll use structured CSV files to build our movie knowledge graph:
- `movies.csv`: Movie nodes with properties
- `people.csv`: Person nodes (actors, directors) 
- `genres.csv`: Genre nodes
- `acted_in.csv`: Actor-Movie relationships
- `directed.csv`: Director-Movie relationships  
- `movie_genres.csv`: Movie-Genre relationships

### 💡 Loading Strategy
1. **Clean existing data** (if needed)
2. **Load nodes first** (movies, people, genres)
3. **Create relationships** between existing nodes
4. **Verify data integrity**

Let's examine our data files first:

In [ ]:
# Examine our CSV data
print("📂 Data File Overview:")
print("=" * 60)

data_files = {
    "Movies": "data/movies.csv",
    "People": "data/people.csv", 
    "Genres": "data/genres.csv",
    "Acting Relationships": "data/acted_in.csv",
    "Directing Relationships": "data/directed.csv",
    "Movie-Genre Relationships": "data/movie_genres.csv"
}

for name, file_path in data_files.items():
    try:
        df = pd.read_csv(file_path)
        print(f"\n📊 {name} ({file_path}):")
        print(f"   Rows: {len(df)}, Columns: {list(df.columns)}")
        print(f"   Sample: {df.iloc[0].to_dict() if len(df) > 0 else 'No data'}")
    except FileNotFoundError:
        print(f"\n❌ {name}: File not found - {file_path}")

### 🧹 Clear Existing Data (Optional)
Before loading new data, let's clean our database to start fresh:

In [ ]:
# CAUTION: This will delete all data in your graph!
# Uncomment the lines below if you want to start with a clean database

# print("⚠️  Clearing all data from the database...")
# kg.query("MATCH (n) DETACH DELETE n")
# print("✅ Database cleared successfully!")

print("ℹ️  Skipping database clear. Remove comments above to clear data.")

### 📥 Loading Node Data

We'll use Cypher's `LOAD CSV` clause to efficiently import data:

In [ ]:
# Load Movie nodes
print("🎬 Loading Movies...")

cypher_movies = """
LOAD CSV WITH HEADERS FROM 'file:///data/movies.csv' AS row
MERGE (m:Movie {title: row.title})
SET m.released = toInteger(row.released),
    m.tagline = row.tagline
RETURN count(m) AS moviesCreated
"""

# Note: For local files, we need to use file:// protocol
# Alternative approach using manual data loading:

movies_df = pd.read_csv('data/movies.csv')
movies_data = movies_df.to_dict('records')

cypher_movies_manual = """
UNWIND $movies AS movie
MERGE (m:Movie {title: movie.title})
SET m.released = movie.released,
    m.tagline = movie.tagline
RETURN count(m) AS moviesCreated
"""

result = kg.query(cypher_movies_manual, params={"movies": movies_data})
print(f"✅ Created {result[0]['moviesCreated']} movies")

In [ ]:
# Load People nodes (actors and directors)
print("👥 Loading People...")

people_df = pd.read_csv('data/people.csv')
people_data = people_df.to_dict('records')

cypher_people = """
UNWIND $people AS person
MERGE (p:Person {name: person.name})
SET p.born = person.born
RETURN count(p) AS peopleCreated
"""

result = kg.query(cypher_people, params={"people": people_data})
print(f"✅ Created {result[0]['peopleCreated']} people")

In [ ]:
# Load Genre nodes
print("🎭 Loading Genres...")

genres_df = pd.read_csv('data/genres.csv')
genres_data = genres_df.to_dict('records')

cypher_genres = """
UNWIND $genres AS genre
MERGE (g:Genre {name: genre.name})
RETURN count(g) AS genresCreated
"""

result = kg.query(cypher_genres, params={"genres": genres_data})
print(f"✅ Created {result[0]['genresCreated']} genres")

### 🔗 Creating Relationships

Now we'll connect our nodes with meaningful relationships:

In [ ]:
# Create ACTED_IN relationships
print("🎭 Creating ACTED_IN relationships...")

acted_in_df = pd.read_csv('data/acted_in.csv')
acted_in_data = acted_in_df.to_dict('records')

cypher_acted_in = """
UNWIND $relationships AS rel
MATCH (a:Person {name: rel.actor})
MATCH (m:Movie {title: rel.movie})
MERGE (a)-[r:ACTED_IN]->(m)
SET r.role = rel.role
RETURN count(r) AS relationshipsCreated
"""

result = kg.query(cypher_acted_in, params={"relationships": acted_in_data})
print(f"✅ Created {result[0]['relationshipsCreated']} ACTED_IN relationships")

In [ ]:
# Create DIRECTED relationships
print("🎬 Creating DIRECTED relationships...")

directed_df = pd.read_csv('data/directed.csv')
directed_data = directed_df.to_dict('records')

cypher_directed = """
UNWIND $relationships AS rel
MATCH (d:Person {name: rel.director})
MATCH (m:Movie {title: rel.movie})
MERGE (d)-[r:DIRECTED]->(m)
RETURN count(r) AS relationshipsCreated
"""

result = kg.query(cypher_directed, params={"relationships": directed_data})
print(f"✅ Created {result[0]['relationshipsCreated']} DIRECTED relationships")

In [ ]:
# Create Movie-Genre relationships
print("🏷️ Creating BELONGS_TO relationships...")

movie_genres_df = pd.read_csv('data/movie_genres.csv')
movie_genres_data = movie_genres_df.to_dict('records')

cypher_movie_genres = """
UNWIND $relationships AS rel
MATCH (m:Movie {title: rel.movie})
MATCH (g:Genre {name: rel.genre})
MERGE (m)-[r:BELONGS_TO]->(g)
RETURN count(r) AS relationshipsCreated
"""

result = kg.query(cypher_movie_genres, params={"relationships": movie_genres_data})
print(f"✅ Created {result[0]['relationshipsCreated']} BELONGS_TO relationships")

### ✅ Data Verification

Let's verify our data was loaded correctly:

In [ ]:
# Comprehensive data verification
print("🔍 Data Verification Report")
print("=" * 50)

# Node counts by label
result = kg.query("""
MATCH (n)
RETURN labels(n)[0] AS label, count(n) AS count
ORDER BY count DESC
""")

print("📊 Node Summary:")
total_nodes = 0
for row in result:
    if row['label']:  # Skip unlabeled nodes
        print(f"   {row['label']}: {row['count']}")
        total_nodes += row['count']
print(f"   TOTAL: {total_nodes}")

# Relationship counts by type
result = kg.query("""
MATCH ()-[r]->()
RETURN type(r) AS relationshipType, count(r) AS count
ORDER BY count DESC
""")

print("\n🔗 Relationship Summary:")
total_relationships = 0
for row in result:
    print(f"   {row['relationshipType']}: {row['count']}")
    total_relationships += row['count']
print(f"   TOTAL: {total_relationships}")

# Sample data verification
print("\n🔍 Sample Data:")
result = kg.query("""
MATCH (p:Person)-[r:ACTED_IN]->(m:Movie)
RETURN p.name AS actor, r.role AS role, m.title AS movie
LIMIT 3
""")

for row in result:
    print(f"   {row['actor']} played {row['role']} in '{row['movie']}'")

## 6. Basic Cypher Queries

Now that we have data loaded, let's explore fundamental Cypher patterns:

### 🔍 Pattern 1: Simple Node Matching
The most basic operation is finding nodes by their properties.

In [ ]:
print("🔍 Basic Node Queries")
print("=" * 40)

# 1. Find all movies
print("\n1️⃣ All movies:")
result = kg.query("""
MATCH (m:Movie)
RETURN m.title AS title, m.released AS year
ORDER BY m.released DESC
LIMIT 5
""")
for movie in result:
    print(f"   📽️ {movie['title']} ({movie['year']})")

# 2. Find movies by specific criteria
print("\n2️⃣ Movies from the 1990s:")
result = kg.query("""
MATCH (m:Movie)
WHERE m.released >= 1990 AND m.released < 2000
RETURN m.title AS title, m.released AS year
ORDER BY m.released
""")
for movie in result:
    print(f"   📽️ {movie['title']} ({movie['year']})")

# 3. Find a specific person
print("\n3️⃣ Information about Keanu Reeves:")
result = kg.query("""
MATCH (p:Person {name: "Keanu Reeves"})
RETURN p.name AS name, p.born AS birthYear
""")
if result:
    person = result[0]
    print(f"   👤 {person['name']}, born {person['birthYear']}")
else:
    print("   ❌ Person not found")

### 🔗 Pattern 2: Relationship Traversal
The power of graphs lies in following relationships between entities.

In [ ]:
print("🔗 Relationship Traversal Queries")
print("=" * 45)

# 1. Find movies that Keanu Reeves acted in
print("\n1️⃣ Keanu Reeves' movies:")
result = kg.query("""
MATCH (p:Person {name: "Keanu Reeves"})-[r:ACTED_IN]->(m:Movie)
RETURN m.title AS movie, r.role AS role, m.released AS year
ORDER BY m.released
""")
for movie in result:
    print(f"   🎬 {movie['movie']} ({movie['year']}) as {movie['role']}")

# 2. Find directors and their movies
print("\n2️⃣ Christopher Nolan's directed movies:")
result = kg.query("""
MATCH (d:Person {name: "Christopher Nolan"})-[:DIRECTED]->(m:Movie)
RETURN m.title AS movie, m.released AS year
ORDER BY m.released
""")
for movie in result:
    print(f"   🎭 {movie['movie']} ({movie['year']})")

# 3. Find movies by genre
print("\n3️⃣ Sci-Fi movies:")
result = kg.query("""
MATCH (m:Movie)-[:BELONGS_TO]->(g:Genre {name: "Sci-Fi"})
RETURN m.title AS movie, m.released AS year
ORDER BY m.released DESC
""")
for movie in result:
    print(f"   🚀 {movie['movie']} ({movie['year']})")

### 🌐 Pattern 3: Multi-hop Relationships
Graph databases excel at finding connections across multiple relationships.

In [ ]:
print("🌐 Multi-hop Relationship Queries")
print("=" * 45)

# 1. Find co-actors (actors who worked in the same movie)
print("\n1️⃣ Keanu Reeves' co-actors:")
result = kg.query("""
MATCH (keanu:Person {name: "Keanu Reeves"})-[:ACTED_IN]->(m:Movie)<-[:ACTED_IN]-(coactor:Person)
WHERE keanu <> coactor  // Exclude Keanu himself
RETURN coactor.name AS coactor, m.title AS movie
""")
for row in result:
    print(f"   🤝 {row['coactor']} in '{row['movie']}'")

# 2. Find actors who worked with specific directors
print("\n2️⃣ Actors who worked with Christopher Nolan:")
result = kg.query("""
MATCH (d:Person {name: "Christopher Nolan"})-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(a:Person)
RETURN DISTINCT a.name AS actor, count(m) AS movieCount, collect(m.title) AS movies
ORDER BY movieCount DESC
""")
for row in result:
    movies_str = ", ".join(row['movies'][:3])  # Show first 3 movies
    if len(row['movies']) > 3:
        movies_str += "..."
    print(f"   🎭 {row['actor']}: {row['movieCount']} movie(s) - {movies_str}")

# 3. Find movies that share multiple genres
print("\n3️⃣ Movies similar to 'The Matrix' (by genre):")
result = kg.query("""
MATCH (matrix:Movie {title: "The Matrix"})-[:BELONGS_TO]->(g:Genre)<-[:BELONGS_TO]-(similar:Movie)
WHERE matrix <> similar
WITH similar, count(g) AS sharedGenres, collect(g.name) AS genres
RETURN similar.title AS movie, similar.released AS year, sharedGenres, genres
ORDER BY sharedGenres DESC, similar.released DESC
""")
for row in result:
    genres_str = ", ".join(row['genres'])
    print(f"   📽️ {row['movie']} ({row['year']}) - {row['sharedGenres']} shared: {genres_str}")

## 7. Data Aggregation and Analysis

### 📊 Aggregation Functions
Cypher provides powerful aggregation functions for data analysis:

| Function | Purpose | Example |
|----------|---------|--------|
| `count()` | Count items | `count(n)`, `count(DISTINCT n.property)` |
| `collect()` | Gather into list | `collect(n.name)` |
| `sum()` | Sum numeric values | `sum(n.age)` |
| `avg()` | Average values | `avg(n.rating)` |
| `min()`, `max()` | Minimum/Maximum | `min(n.released)`, `max(n.released)` |
| `size()` | Collection size | `size(collect(n))` |

In [ ]:
print("📊 Data Aggregation and Analysis")
print("=" * 45)

# 1. Most prolific actors (by number of movies)
print("\n1️⃣ Most prolific actors:")
result = kg.query("""
MATCH (a:Person)-[:ACTED_IN]->(m:Movie)
WITH a, count(m) AS movieCount, collect(m.title) AS movies
WHERE movieCount >= 1  // At least 1 movie
RETURN a.name AS actor, movieCount, movies
ORDER BY movieCount DESC, a.name
LIMIT 5
""")
for row in result:
    movies_preview = ", ".join(row['movies'][:2])  # Show first 2 movies
    if len(row['movies']) > 2:
        movies_preview += "..."
    print(f"   🏆 {row['actor']}: {row['movieCount']} movies ({movies_preview})")

# 2. Most popular genres (by number of movies)
print("\n2️⃣ Most popular genres:")
result = kg.query("""
MATCH (g:Genre)<-[:BELONGS_TO]-(m:Movie)
RETURN g.name AS genre, count(m) AS movieCount
ORDER BY movieCount DESC
""")
for row in result:
    print(f"   📊 {row['genre']}: {row['movieCount']} movies")

# 3. Movies by decade
print("\n3️⃣ Movies by decade:")
result = kg.query("""
MATCH (m:Movie)
WITH (m.released / 10) * 10 AS decade, count(m) AS movieCount
RETURN decade, movieCount
ORDER BY decade DESC
""")
for row in result:
    print(f"   📅 {int(row['decade'])}s: {row['movieCount']} movies")

# 4. Average career span analysis
print("\n4️⃣ Directors with multiple movies (career span):")
result = kg.query("""
MATCH (d:Person)-[:DIRECTED]->(m:Movie)
WITH d, count(m) AS movieCount, 
     min(m.released) AS firstMovie, 
     max(m.released) AS lastMovie,
     collect(m.title) AS movies
WHERE movieCount > 1
RETURN d.name AS director, movieCount, 
       firstMovie, lastMovie, 
       (lastMovie - firstMovie) AS careerSpan,
       movies
ORDER BY careerSpan DESC
""")
for row in result:
    movies_str = ", ".join(row['movies'])
    print(f"   🎬 {row['director']}: {row['movieCount']} movies over {row['careerSpan']} years")
    print(f"       ({row['firstMovie']}-{row['lastMovie']}) - {movies_str}")

## 8. Advanced Cypher Patterns

### 🧠 Complex Query Patterns
Let's explore more sophisticated graph analysis techniques:

In [ ]:
print("🧠 Advanced Cypher Patterns")
print("=" * 40)

# 1. Recommendation Engine: Find movies liked by people with similar tastes
print("\n1️⃣ Movie Recommendations for Sci-Fi fans:")
print("   (Movies that share genres with highly-rated Sci-Fi films)")
result = kg.query("""
// Find genres that appear in Sci-Fi movies
MATCH (scifi:Genre {name: "Sci-Fi"})<-[:BELONGS_TO]-(scifiMovie:Movie)-[:BELONGS_TO]->(sharedGenre:Genre)
WHERE sharedGenre.name <> "Sci-Fi"  // Exclude Sci-Fi itself

// Find other movies with these shared genres
MATCH (sharedGenre)<-[:BELONGS_TO]-(recommendedMovie:Movie)
WHERE NOT (recommendedMovie)-[:BELONGS_TO]->(scifi)  // Exclude pure Sci-Fi movies

// Count connections and return recommendations
WITH recommendedMovie, count(DISTINCT sharedGenre) AS genreOverlap, collect(DISTINCT sharedGenre.name) AS sharedGenres
RETURN recommendedMovie.title AS movie, recommendedMovie.released AS year, 
       genreOverlap, sharedGenres
ORDER BY genreOverlap DESC, recommendedMovie.released DESC
LIMIT 5
""")
for row in result:
    genres_str = ", ".join(row['sharedGenres'])
    print(f"   💡 {row['movie']} ({row['year']}) - {row['genreOverlap']} shared genres: {genres_str}")

# 2. Social Network Analysis: Find the "Kevin Bacon" of our graph
print("\n2️⃣ Most Connected Actor (Graph centrality):")
result = kg.query("""
MATCH (actor:Person)-[:ACTED_IN]->(m:Movie)<-[:ACTED_IN]-(coactor:Person)
WHERE actor <> coactor
WITH actor, count(DISTINCT coactor) AS connections, count(DISTINCT m) AS movies
RETURN actor.name AS actor, connections, movies, 
       round(toFloat(connections) / movies, 2) AS avgCoactorsPerMovie
ORDER BY connections DESC
LIMIT 5
""")
for row in result:
    print(f"   🌟 {row['actor']}: {row['connections']} co-actors across {row['movies']} movies")
    print(f"       Average {row['avgCoactorsPerMovie']} co-actors per movie")

# 3. Find "Bridge" Movies (connect different genres/communities)
print("\n3️⃣ Bridge Movies (connecting multiple genres):")
result = kg.query("""
MATCH (m:Movie)-[:BELONGS_TO]->(g:Genre)
WITH m, count(g) AS genreCount, collect(g.name) AS genres
WHERE genreCount >= 3  // Movies with 3+ genres
RETURN m.title AS movie, m.released AS year, genreCount, genres
ORDER BY genreCount DESC, m.released DESC
""")
for row in result:
    genres_str = ", ".join(row['genres'])
    print(f"   🌉 {row['movie']} ({row['year']}) - {row['genreCount']} genres: {genres_str}")

### 🔀 Conditional Logic and Pattern Matching
Advanced pattern matching with conditional logic:

In [ ]:
print("🔀 Conditional Logic and Advanced Patterns")
print("=" * 50)

# 1. OPTIONAL MATCH - Handle missing relationships gracefully
print("\n1️⃣ People and their roles (actors vs directors):")
result = kg.query("""
MATCH (p:Person)
OPTIONAL MATCH (p)-[acted:ACTED_IN]->(actedMovie:Movie)
OPTIONAL MATCH (p)-[directed:DIRECTED]->(directedMovie:Movie)
WITH p, 
     count(DISTINCT actedMovie) AS actedCount,
     count(DISTINCT directedMovie) AS directedCount,
     collect(DISTINCT actedMovie.title)[0..2] AS sampleActed,
     collect(DISTINCT directedMovie.title)[0..2] AS sampleDirected
WHERE actedCount > 0 OR directedCount > 0  // Has at least one relationship
RETURN p.name AS person,
       CASE 
         WHEN actedCount > 0 AND directedCount > 0 THEN "Actor-Director"
         WHEN actedCount > 0 THEN "Actor"
         WHEN directedCount > 0 THEN "Director"
         ELSE "Unknown"
       END AS role,
       actedCount, directedCount, sampleActed, sampleDirected
ORDER BY (actedCount + directedCount) DESC
LIMIT 8
""")
for row in result:
    print(f"   👤 {row['person']} ({row['role']})")
    if row['actedCount'] > 0:
        acted_sample = ", ".join([m for m in row['sampleActed'] if m])
        print(f"       Acted in {row['actedCount']} movies: {acted_sample}...")
    if row['directedCount'] > 0:
        directed_sample = ", ".join([m for m in row['sampleDirected'] if m])
        print(f"       Directed {row['directedCount']} movies: {directed_sample}...")

# 2. Variable Length Paths - Find connections within N steps
print("\n2️⃣ Find connection paths between actors (within 3 steps):")
result = kg.query("""
MATCH path = (start:Person {name: "Keanu Reeves"})-[*1..3]-(end:Person {name: "Al Pacino"})
WHERE start <> end
RETURN [node IN nodes(path) | 
        CASE 
          WHEN 'Person' IN labels(node) THEN node.name
          WHEN 'Movie' IN labels(node) THEN node.title
          ELSE 'Unknown'
        END
       ] AS connectionPath,
       length(path) AS pathLength
ORDER BY pathLength
LIMIT 3
""")
print("   Connection paths from Keanu Reeves to Al Pacino:")
for i, row in enumerate(result):
    path_str = " → ".join(row['connectionPath'])
    print(f"   {i+1}. {path_str} (length: {row['pathLength']})")
    
if not result:
    print("   ❌ No connection found within 3 steps")

# 3. Existence patterns - Movies without certain relationships
print("\n3️⃣ Movies missing genre information:")
result = kg.query("""
MATCH (m:Movie)
WHERE NOT (m)-[:BELONGS_TO]->(:Genre)
RETURN m.title AS movie, m.released AS year
ORDER BY m.released DESC
""")
if result:
    for row in result:
        print(f"   ❓ {row['movie']} ({row['year']}) - No genre assigned")
else:
    print("   ✅ All movies have genre information!")

## 9. Graph Analysis and Insights

### 📈 Graph Metrics and Analysis
Let's analyze our graph structure and discover insights:

In [ ]:
print("📈 Graph Structure Analysis")
print("=" * 40)

# 1. Graph density and connectivity
print("\n1️⃣ Graph Statistics:")
result = kg.query("""
MATCH (n)
OPTIONAL MATCH (n)-[r]-()
WITH labels(n)[0] AS nodeType, count(DISTINCT n) AS nodeCount, count(DISTINCT r) AS relationshipCount
RETURN nodeType, nodeCount, relationshipCount
ORDER BY nodeCount DESC
""")
total_nodes = sum([row['nodeCount'] for row in result if row['nodeType']])
total_relationships = kg.query("MATCH ()-[r]->() RETURN count(r) AS total")[0]['total']

print(f"   📊 Total nodes: {total_nodes}")
print(f"   🔗 Total relationships: {total_relationships}")
print(f"   📏 Average degree: {round(total_relationships * 2 / total_nodes, 2)}")

# 2. Most influential nodes (highest degree centrality)
print("\n2️⃣ Most Connected Entities:")
result = kg.query("""
MATCH (n)-[r]-()
WITH n, count(r) AS degree, labels(n)[0] AS nodeType
RETURN nodeType, 
       CASE nodeType
         WHEN 'Person' THEN n.name
         WHEN 'Movie' THEN n.title
         WHEN 'Genre' THEN n.name
         ELSE 'Unknown'
       END AS name,
       degree
ORDER BY degree DESC
LIMIT 8
""")
for row in result:
    print(f"   🌟 {row['nodeType']}: {row['name']} (degree: {row['degree']})")

# 3. Community detection - Find tightly connected groups
print("\n3️⃣ Movie Clusters by Shared Actors:")
result = kg.query("""
// Find movies that share at least 2 actors
MATCH (m1:Movie)<-[:ACTED_IN]-(a:Person)-[:ACTED_IN]->(m2:Movie)
WHERE id(m1) < id(m2)  // Avoid duplicates
WITH m1, m2, count(a) AS sharedActors
WHERE sharedActors >= 2
RETURN m1.title AS movie1, m2.title AS movie2, sharedActors
ORDER BY sharedActors DESC
""")
if result:
    for row in result:
        print(f"   🎭 '{row['movie1']}' ↔ '{row['movie2']}': {row['sharedActors']} shared actors")
else:
    print("   ❌ No movies share multiple actors")

# 4. Shortest path analysis
print("\n4️⃣ Six Degrees of Separation (Actor Network):")
result = kg.query("""
MATCH (a1:Person)-[:ACTED_IN]->(:Movie)<-[:ACTED_IN]-(a2:Person)
WHERE a1 <> a2
WITH a1, a2, 1 AS distance
RETURN distance, count(*) AS pairs
UNION ALL
MATCH (a1:Person)-[:ACTED_IN]->(:Movie)<-[:ACTED_IN]-(:Person)-[:ACTED_IN]->(:Movie)<-[:ACTED_IN]-(a2:Person)
WHERE a1 <> a2
WITH a1, a2, 2 AS distance
RETURN distance, count(*) AS pairs
ORDER BY distance
""")
for row in result:
    print(f"   🔗 {row['distance']} degree(s) of separation: {row['pairs']} actor pairs")

### 🔍 Data Quality and Validation
Let's check our data quality and find potential issues:

In [ ]:
print("🔍 Data Quality Analysis")
print("=" * 35)

# 1. Find orphaned nodes (nodes without relationships)
print("\n1️⃣ Orphaned Nodes (no relationships):")
result = kg.query("""
MATCH (n)
WHERE NOT (n)-[]-() 
RETURN labels(n)[0] AS nodeType, count(n) AS orphanCount
ORDER BY orphanCount DESC
""")
if result and any(row['orphanCount'] > 0 for row in result):
    for row in result:
        if row['orphanCount'] > 0:
            print(f"   ⚠️ {row['nodeType']}: {row['orphanCount']} orphaned nodes")
else:
    print("   ✅ No orphaned nodes found!")

# 2. Find duplicate or similar names
print("\n2️⃣ Potential Data Quality Issues:")
result = kg.query("""
MATCH (p:Person)
WITH p.name AS name, count(p) AS duplicateCount
WHERE duplicateCount > 1
RETURN name, duplicateCount
""")
if result:
    print("   Duplicate person names:")
    for row in result:
        print(f"   ⚠️ '{row['name']}' appears {row['duplicateCount']} times")
else:
    print("   ✅ No duplicate person names")

# 3. Missing critical properties
print("\n3️⃣ Missing Properties:")
result = kg.query("""
MATCH (m:Movie)
WHERE m.released IS NULL OR m.tagline IS NULL
RETURN m.title AS movie, 
       CASE WHEN m.released IS NULL THEN 'Missing release year' ELSE '' END +
       CASE WHEN m.tagline IS NULL THEN 'Missing tagline' ELSE '' END AS issues
""")
if result:
    for row in result:
        if row['issues']:
            print(f"   ⚠️ {row['movie']}: {row['issues']}")
else:
    print("   ✅ All movies have complete property data")

# 4. Relationship consistency check
print("\n4️⃣ Relationship Validation:")
result = kg.query("""
// Check for actors who also directed the same movie
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)<-[:DIRECTED]-(p)
RETURN p.name AS person, m.title AS movie
""")
if result:
    print("   Actor-Directors (acted in their own films):")
    for row in result:
        print(f"   🎭🎬 {row['person']} in '{row['movie']}'")
else:
    print("   ℹ️ No actor-directors found in the same movie")

# 5. Summary statistics
print("\n5️⃣ Data Completeness Summary:")
stats = {
    "Total Movies": kg.query("MATCH (m:Movie) RETURN count(m) AS count")[0]['count'],
    "Total People": kg.query("MATCH (p:Person) RETURN count(p) AS count")[0]['count'],
    "Total Genres": kg.query("MATCH (g:Genre) RETURN count(g) AS count")[0]['count'],
    "Acting Relationships": kg.query("MATCH ()-[r:ACTED_IN]->() RETURN count(r) AS count")[0]['count'],
    "Directing Relationships": kg.query("MATCH ()-[r:DIRECTED]->() RETURN count(r) AS count")[0]['count'],
    "Genre Relationships": kg.query("MATCH ()-[r:BELONGS_TO]->() RETURN count(r) AS count")[0]['count']
}

for metric, value in stats.items():
    print(f"   📊 {metric}: {value}")

## 10. Practical Applications and Use Cases

### 🎯 Real-World Scenarios
Let's explore how these graph concepts apply to real business problems:

In [ ]:
print("🎯 Practical Applications")
print("=" * 35)

# 1. Content Recommendation System
print("\n1️⃣ Content Recommendation Engine:")
print("   Scenario: User likes 'The Matrix' - what should we recommend?")

result = kg.query("""
// Multi-factor recommendation based on:
// 1. Shared genres
// 2. Shared actors
// 3. Similar release timeframe
MATCH (liked:Movie {title: "The Matrix"})

// Find movies with shared characteristics
MATCH (candidate:Movie)
WHERE candidate <> liked

// Calculate genre similarity
OPTIONAL MATCH (liked)-[:BELONGS_TO]->(sharedGenre:Genre)<-[:BELONGS_TO]-(candidate)
WITH candidate, liked, count(DISTINCT sharedGenre) AS genreScore

// Calculate actor similarity
OPTIONAL MATCH (liked)<-[:ACTED_IN]-(sharedActor:Person)-[:ACTED_IN]->(candidate)
WITH candidate, liked, genreScore, count(DISTINCT sharedActor) AS actorScore

// Calculate time proximity (prefer movies within 5 years)
WITH candidate, liked, genreScore, actorScore,
     CASE WHEN abs(candidate.released - liked.released) <= 5 THEN 1 ELSE 0 END AS timeScore

// Calculate overall recommendation score
WITH candidate, (genreScore * 2 + actorScore * 3 + timeScore) AS recommendationScore,
     genreScore, actorScore, timeScore
WHERE recommendationScore > 0

RETURN candidate.title AS movie, candidate.released AS year,
       recommendationScore, genreScore, actorScore, timeScore
ORDER BY recommendationScore DESC, candidate.released DESC
LIMIT 5
""")

for row in result:
    print(f"   🎬 {row['movie']} ({row['year']}) - Score: {row['recommendationScore']}")
    print(f"       Genre: {row['genreScore']}, Actor: {row['actorScore']}, Time: {row['timeScore']}")

# 2. Influencer Identification
print("\n2️⃣ Influencer Analysis (Key People in Network):")
result = kg.query("""
// Find people who connect different movie communities
MATCH (p:Person)-[:ACTED_IN|DIRECTED]->(m:Movie)-[:BELONGS_TO]->(g:Genre)
WITH p, count(DISTINCT m) AS movieCount, 
     count(DISTINCT g) AS genreSpread,
     collect(DISTINCT g.name) AS genres
WHERE movieCount >= 1 AND genreSpread >= 2
RETURN p.name AS person, movieCount, genreSpread, genres,
       round(toFloat(genreSpread) / movieCount, 2) AS diversityScore
ORDER BY genreSpread DESC, movieCount DESC
LIMIT 5
""")

for row in result:
    genres_str = ", ".join(row['genres'][:4])  # Show first 4 genres
    print(f"   🌟 {row['person']}: {row['movieCount']} movies across {row['genreSpread']} genres")
    print(f"       Diversity score: {row['diversityScore']} - Genres: {genres_str}")

# 3. Market Segmentation
print("\n3️⃣ Market Segmentation (Genre Preferences by Era):")
result = kg.query("""
MATCH (m:Movie)-[:BELONGS_TO]->(g:Genre)
WITH (m.released / 10) * 10 AS decade, g.name AS genre, count(m) AS movieCount
WHERE decade >= 1990  // Focus on recent decades
RETURN decade, genre, movieCount
ORDER BY decade DESC, movieCount DESC
""")

# Group by decade for better presentation
decades = {}
for row in result:
    decade = int(row['decade'])
    if decade not in decades:
        decades[decade] = []
    decades[decade].append((row['genre'], row['movieCount']))

for decade in sorted(decades.keys(), reverse=True):
    print(f"   📅 {decade}s - Top genres:")
    for genre, count in decades[decade][:3]:  # Top 3 per decade
        print(f"       {genre}: {count} movies")

# 4. Risk Assessment (Single Points of Failure)
print("\n4️⃣ Risk Assessment - Critical Dependencies:")
result = kg.query("""
// Find genres that would disappear if we lost certain movies
MATCH (g:Genre)<-[:BELONGS_TO]-(m:Movie)
WITH g, count(m) AS movieCount, collect(m.title) AS movies
WHERE movieCount = 1  // Genres with only one movie
RETURN g.name AS vulnerableGenre, movies[0] AS singleMovie
""")

if result:
    print("   ⚠️ Vulnerable genres (dependent on single movies):")
    for row in result:
        print(f"       {row['vulnerableGenre']} → only in '{row['singleMovie']}'")
else:
    print("   ✅ All genres are represented in multiple movies")

print("\n💡 Business Insights Summary:")
print("   • Recommendation systems can leverage multiple relationship types")
print("   • Network analysis reveals influential people and content")
print("   • Trend analysis shows market evolution over time")
print("   • Risk assessment identifies critical dependencies")

## 11. Performance Tips and Best Practices

### ⚡ Query Optimization

#### 🎯 Best Practices for Cypher Queries:

1. **Use Labels and Indexes**
   ```cypher
   // Good: Use specific labels
   MATCH (p:Person {name: "Tom Hanks"})
   
   // Avoid: Scanning all nodes
   MATCH (p {name: "Tom Hanks"})
   ```

2. **Start with Most Selective Patterns**
   ```cypher
   // Good: Start with specific node
   MATCH (specific:Movie {title: "The Matrix"})-[:BELONGS_TO]->(g:Genre)
   
   // Avoid: Start with broad pattern
   MATCH (g:Genre)<-[:BELONGS_TO]-(m:Movie {title: "The Matrix"})
   ```

3. **Use LIMIT Early**
   ```cypher
   // Good: Limit early in processing
   MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
   WITH p, count(m) AS movieCount
   ORDER BY movieCount DESC
   LIMIT 10
   RETURN p.name, movieCount
   ```

#### 📊 Index Creation
For production systems, create indexes on frequently queried properties:

In [ ]:
print("⚡ Performance Tips and Index Management")
print("=" * 50)

# 1. Check existing indexes
print("\n1️⃣ Current Database Indexes:")
try:
    result = kg.query("SHOW INDEXES")
    if result:
        for idx in result:
            print(f"   📇 {idx.get('name', 'unnamed')}: {idx.get('labelsOrTypes', [])} - {idx.get('properties', [])}")
    else:
        print("   ℹ️ No indexes found or command not supported")
except Exception as e:
    print(f"   ⚠️ Could not retrieve indexes: {str(e)[:100]}...")

# 2. Query performance demonstration
print("\n2️⃣ Query Performance Examples:")

# Example of efficient vs inefficient queries
import time

# Efficient query - using labels and specific properties
start_time = time.time()
result = kg.query("""
MATCH (p:Person {name: "Keanu Reeves"})-[:ACTED_IN]->(m:Movie)
RETURN count(m) AS movieCount
""")
efficient_time = time.time() - start_time

# Less efficient query - broader pattern
start_time = time.time()
result2 = kg.query("""
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name = "Keanu Reeves"
RETURN count(m) AS movieCount
""")
less_efficient_time = time.time() - start_time

print(f"   🏃‍♂️ Efficient query (with node property): {efficient_time:.4f} seconds")
print(f"   🚶‍♂️ Less efficient query (with WHERE): {less_efficient_time:.4f} seconds")

# 3. Memory-conscious queries
print("\n3️⃣ Memory-Efficient Patterns:")
print("   💡 Use LIMIT and pagination for large result sets")
print("   💡 Avoid collecting large lists - use aggregation instead")
print("   💡 Use EXPLAIN/PROFILE to analyze query execution")

# Example of memory-conscious aggregation
result = kg.query("""
// Memory-efficient: Count instead of collecting
MATCH (g:Genre)<-[:BELONGS_TO]-(m:Movie)
RETURN g.name AS genre, count(m) AS movieCount
ORDER BY movieCount DESC
LIMIT 3
""")
print("\n   📊 Top genres (memory-efficient counting):")
for row in result:
    print(f"       {row['genre']}: {row['movieCount']} movies")

# 4. Best practices summary
print("\n4️⃣ Query Optimization Checklist:")
best_practices = [
    "✅ Always use node labels in MATCH clauses",
    "✅ Start patterns with most selective nodes", 
    "✅ Use indexes on frequently queried properties",
    "✅ Apply LIMIT early in query processing",
    "✅ Use DISTINCT only when necessary",
    "✅ Prefer aggregation over large collections",
    "✅ Use OPTIONAL MATCH for optional relationships",
    "✅ Profile queries with EXPLAIN/PROFILE"
]

for practice in best_practices:
    print(f"   {practice}")

## 12. Conclusion and Next Steps

### 🎉 What You've Learned

Congratulations! You've completed a comprehensive tour of Knowledge Graphs and Cypher. Here's what you've mastered:

#### 📚 **Conceptual Understanding**
- ✅ Knowledge Graph fundamentals and use cases
- ✅ Graph vs. relational database concepts
- ✅ Neo4j architecture and labeled property graphs

#### 🔧 **Technical Skills**
- ✅ Cypher query language syntax and patterns
- ✅ Data loading from CSV files
- ✅ Basic to advanced graph traversals
- ✅ Aggregation and analytical queries
- ✅ Performance optimization techniques

#### 🎯 **Practical Applications**
- ✅ Recommendation systems
- ✅ Social network analysis
- ✅ Data quality assessment
- ✅ Business intelligence and insights

### 🚀 **Next Steps**

#### **Immediate Actions:**
1. **Practice**: Try modifying the queries in this notebook
2. **Experiment**: Load your own CSV data into the graph
3. **Explore**: Try the Neo4j Browser interface for visual exploration

#### **Advanced Topics to Explore:**
- **Graph Data Science Library (GDS)**: Advanced algorithms for centrality, community detection, similarity
- **APOC Procedures**: Extended functionality for data processing and analysis
- **Graph Neural Networks**: Machine learning on graph data
- **Multi-database setups**: Sharding and federation
- **Real-time graph streaming**: Kafka integration and live updates

#### **Resources for Continued Learning:**
- 📖 [Neo4j Documentation](https://neo4j.com/docs/)
- 🎓 [Neo4j GraphAcademy](https://graphacademy.neo4j.com/)
- 👥 [Neo4j Community Forum](https://community.neo4j.com/)
- 📊 [Graph Data Science Playground](https://neo4j.com/sandbox/)

### 💡 **Final Tips**
- **Think in Relationships**: Always consider how your entities connect
- **Start Simple**: Begin with basic patterns and build complexity gradually
- **Visualize**: Use Neo4j Browser or Bloom to explore your graphs visually
- **Performance Matters**: Always consider indexing and query optimization
- **Community**: Join the Neo4j community - it's incredibly helpful and welcoming!

---

### 🙏 **Thank You!**
You've completed the Knowledge Graphs and Cypher tutorial. You now have the foundation to build powerful graph-based applications and analyses. The world of connected data awaits your exploration!

**Happy Graphing!** 🎊

## 14. GraphRAG Integration - Financial Investment Advisor\n\n### 🏦 Building a Real-World Financial Analysis System\n\nGraphRAG (Graph Retrieval-Augmented Generation) combines the power of Knowledge Graphs with Large Language Models to provide contextual, accurate responses based on structured data relationships.\n\n#### 💡 **What We'll Build:**\nA **Financial Investment Advisor** that:\n- Ingests SEC EDGAR 10-K annual reports from major companies\n- Extracts financial entities, risks, and key metrics\n- Creates a knowledge graph of financial relationships\n- Uses vector embeddings for semantic search\n- Provides AI-powered investment recommendations\n\n#### 🎯 **Business Impact:**\n- **Investment Decision Support**: Data-driven investment recommendations\n- **Risk Analysis**: Identify and assess financial risks across portfolios\n- **Market Intelligence**: Understand company relationships and dependencies\n- **Compliance**: Track regulatory changes and their business impacts\n\n#### 🛠️ **Technology Stack (All Open Source):**\n- **Data Source**: SEC EDGAR API (free, public financial data)\n- **LLM**: Ollama (Llama 3.1) or HuggingFace Transformers\n- **Embeddings**: sentence-transformers (all-MiniLM-L6-v2)\n- **Graph Database**: Neo4j Aura Free Tier\n- **Vector Search**: Neo4j built-in vector indexing\n\n---"

### 🚀 Setup and Dependencies\n\nFirst, let's install and import the additional libraries needed for our financial GraphRAG system:"

In [ ]:
# Install additional dependencies (run once)\n# !pip install sentence-transformers transformers torch requests beautifulsoup4 lxml yfinance plotly wordcloud\n\n# Import libraries for GraphRAG Financial Advisor\nimport requests\nimport json\nfrom bs4 import BeautifulSoup\nimport re\nfrom datetime import datetime, timedelta\nfrom typing import List, Dict, Any, Optional, Tuple\nfrom dataclasses import dataclass\nimport warnings\nwarnings.filterwarnings('ignore')\n\n# ML and NLP libraries\ntry:\n    from sentence_transformers import SentenceTransformer\n    from transformers import pipeline, AutoTokenizer, AutoModel\n    import torch\n    print(\"✅ ML libraries loaded successfully\")\nexcept ImportError as e:\n    print(f\"⚠️ ML libraries not available: {e}\")\n    print(\"Please run: pip install sentence-transformers transformers torch\")\n\n# Data processing\nimport numpy as np\nimport pandas as pd\nfrom sklearn.metrics.pairwise import cosine_similarity\n\n# Visualization (optional)\ntry:\n    import plotly.graph_objects as go\n    import plotly.express as px\n    from wordcloud import WordCloud\n    import matplotlib.pyplot as plt\n    print(\"✅ Visualization libraries loaded\")\nexcept ImportError:\n    print(\"ℹ️ Visualization libraries not available (optional)\")\n\nprint(\"🔧 GraphRAG Financial Advisor dependencies loaded!\")"

### 📊 SEC EDGAR Data Extraction Pipeline\n\nThe SEC EDGAR database contains comprehensive financial information about public companies. We'll create a pipeline to extract and process 10-K annual reports."

In [ ]:
# SEC EDGAR Data Extractor\n@dataclass\nclass CompanyInfo:\n    \"\"\"Structure for company information\"\"\"\n    cik: str\n    ticker: str\n    name: str\n    sector: str = \"\"\n    industry: str = \"\"\n\n@dataclass \nclass FinancialDocument:\n    \"\"\"Structure for financial documents\"\"\"\n    company: CompanyInfo\n    document_type: str\n    filing_date: str\n    period_end: str\n    content: Dict[str, str]  # section_name: content\n    url: str\n\nclass SECDataExtractor:\n    \"\"\"Extract and process SEC EDGAR financial data\"\"\"\n    \n    def __init__(self):\n        self.base_url = \"https://data.sec.gov\"\n        self.headers = {\n            'User-Agent': 'Neo4j GraphRAG Tutorial (educational@example.com)',\n            'Accept-Encoding': 'gzip, deflate',\n            'Host': 'data.sec.gov'\n        }\n        \n        # Major companies for demonstration (CIK numbers)\n        self.demo_companies = {\n            'AAPL': CompanyInfo('320193', 'AAPL', 'Apple Inc.', 'Technology', 'Consumer Electronics'),\n            'MSFT': CompanyInfo('789019', 'MSFT', 'Microsoft Corporation', 'Technology', 'Software'),\n            'GOOGL': CompanyInfo('1652044', 'GOOGL', 'Alphabet Inc.', 'Technology', 'Internet Services'),\n            'TSLA': CompanyInfo('1318605', 'TSLA', 'Tesla, Inc.', 'Automotive', 'Electric Vehicles'),\n            'AMZN': CompanyInfo('1018724', 'AMZN', 'Amazon.com, Inc.', 'Technology', 'E-commerce')\n        }\n    \n    def get_company_filings(self, cik: str, form_type: str = '10-K', limit: int = 3) -> List[Dict]:\n        \"\"\"Get recent filings for a company\"\"\"\n        url = f\"{self.base_url}/submissions/CIK{cik.zfill(10)}.json\"\n        \n        try:\n            response = requests.get(url, headers=self.headers)\n            response.raise_for_status()\n            data = response.json()\n            \n            # Extract recent filings\n            filings = []\n            recent_filings = data.get('filings', {}).get('recent', {})\n            \n            forms = recent_filings.get('form', [])\n            dates = recent_filings.get('filingDate', [])\n            accession_numbers = recent_filings.get('accessionNumber', [])\n            \n            for i, form in enumerate(forms):\n                if form == form_type and len(filings) < limit:\n                    filings.append({\n                        'form': form,\n                        'filing_date': dates[i],\n                        'accession_number': accession_numbers[i]\n                    })\n            \n            return filings\n            \n        except requests.RequestException as e:\n            print(f\"❌ Error fetching filings for CIK {cik}: {e}\")\n            return []\n    \n    def extract_10k_sections(self, content: str) -> Dict[str, str]:\n        \"\"\"Extract key sections from 10-K filing content\"\"\"\n        sections = {}\n        \n        # Common 10-K sections to extract\n        section_patterns = {\n            'business_overview': r'item\\s*1[\\s\\.].*?business',\n            'risk_factors': r'item\\s*1a[\\s\\.].*?risk\\s*factors',\n            'properties': r'item\\s*2[\\s\\.].*?properties',\n            'legal_proceedings': r'item\\s*3[\\s\\.].*?legal\\s*proceedings',\n            'management_discussion': r'item\\s*7[\\s\\.].*?management.*?discussion',\n            'financial_statements': r'item\\s*8[\\s\\.].*?financial\\s*statements',\n            'controls_procedures': r'item\\s*9a[\\s\\.].*?controls.*?procedures'\n        }\n        \n        content_lower = content.lower()\n        \n        for section_name, pattern in section_patterns.items():\n            try:\n                # Find section start\n                match = re.search(pattern, content_lower, re.IGNORECASE | re.DOTALL)\n                if match:\n                    start_pos = match.start()\n                    \n                    # Find next section or end (simplified extraction)\n                    # In production, you'd want more sophisticated parsing\n                    end_pos = start_pos + 5000  # Take first 5000 chars as sample\n                    section_text = content[start_pos:min(end_pos, len(content))]\n                    \n                    # Clean up the text\n                    section_text = re.sub(r'\\s+', ' ', section_text)  # Normalize whitespace\n                    section_text = section_text.strip()\n                    \n                    if len(section_text) > 100:  # Only include substantial content\n                        sections[section_name] = section_text\n                        \n            except Exception as e:\n                print(f\"⚠️ Error extracting {section_name}: {e}\")\n        \n        return sections\n    \n    def get_demo_financial_data(self) -> List[FinancialDocument]:\n        \"\"\"Get sample financial data for demonstration\"\"\"\n        print(\"📊 Extracting sample financial data for major companies...\")\n        \n        documents = []\n        \n        # For demo purposes, we'll create sample documents with realistic content\n        # In production, you'd fetch real SEC data\n        sample_data = {\n            'AAPL': {\n                'business_overview': \"\"\"\n                Apple Inc. designs, manufactures and markets smartphones, personal computers, tablets, \n                wearables and accessories worldwide. The Company's products include iPhone, Mac, iPad, \n                AirPods, Apple TV, Apple Watch, Beats products, and HomePod. The Company also sells \n                various related services including advertising, AppleCare, cloud services, digital content \n                stores and streaming, and payment services.\n                \"\"\",\n                'risk_factors': \"\"\"\n                The Company is subject to intense competition in all areas of its business, and it competes \n                on price, product features, relative price/performance, product quality and reliability, \n                design innovation, a strong third-party software and accessories ecosystem, marketing and \n                distribution capability, service and support, and corporate reputation. Global markets for \n                the Company's products and services are highly competitive and subject to rapid technological \n                change, and the Company may be unable to compete effectively in these markets.\n                \"\"\",\n                'management_discussion': \"\"\"\n                Net sales increased during 2023 compared to 2022 due to higher net sales of iPhone, Services \n                and Mac, partially offset by lower net sales of iPad and Wearables, Home and Accessories. \n                iPhone net sales increased due to higher sales volume of iPhone 15 models. Services net sales \n                growth was driven by advertising, AppleCare and the App Store.\n                \"\"\"\n            },\n            'MSFT': {\n                'business_overview': \"\"\"\n                Microsoft Corporation develops, licenses, and supports software, services, devices and solutions \n                worldwide. The company operates through three segments: Productivity and Business Processes, \n                Intelligent Cloud, and More Personal Computing. We strive to create local opportunity, growth, \n                and impact in every country around the world.\n                \"\"\",\n                'risk_factors': \"\"\"\n                Security threats and cyberattacks could lead to reduced revenue, increased costs, liability \n                claims, or harm to our reputation or competitive position. Cyberattacks are a critical risk \n                for our company and our customers. We devote substantial resources to defend against security \n                threats, but these measures may be insufficient.\n                \"\"\",\n                'management_discussion': \"\"\"\n                Revenue increased $28.9 billion or 16% driven by growth across our three segments. Productivity \n                and Business Processes revenue increased driven by Microsoft 365 Commercial and Microsoft Teams. \n                Intelligent Cloud revenue increased primarily due to growth in Azure and other cloud services.\n                \"\"\"\n            }\n        }\n        \n        for ticker, company in self.demo_companies.items():\n            if ticker in sample_data:\n                doc = FinancialDocument(\n                    company=company,\n                    document_type='10-K',\n                    filing_date='2023-10-27',\n                    period_end='2023-09-30',\n                    content=sample_data[ticker],\n                    url=f\"https://sec.gov/demo/{ticker}/10-K/2023\"\n                )\n                documents.append(doc)\n        \n        print(f\"✅ Extracted {len(documents)} sample financial documents\")\n        return documents\n\n# Initialize the SEC data extractor\nsec_extractor = SECDataExtractor()\n\n# Get demo financial data\nfinancial_documents = sec_extractor.get_demo_financial_data()\n\nprint(f\"\\n📋 Available Companies:\")\nfor ticker, company in sec_extractor.demo_companies.items():\n    print(f\"   {ticker}: {company.name} ({company.sector})\")\n\nprint(f\"\\n📄 Sample Document Content:\")\nif financial_documents:\n    sample_doc = financial_documents[0]\n    print(f\"Company: {sample_doc.company.name}\")\n    print(f\"Sections: {list(sample_doc.content.keys())}\")\n    print(f\"Business Overview (first 200 chars): {sample_doc.content.get('business_overview', '')[:200]}...\")"

### 🤖 Open Source LLM Integration\n\nWe'll integrate open-source language models for text processing and generation. This setup supports both local Ollama models and HuggingFace models."

In [ ]:
# Open Source LLM Integration\nclass LLMManager:\n    \"\"\"Manager for different LLM backends (Ollama, HuggingFace, etc.)\"\"\"\n    \n    def __init__(self):\n        self.embedding_model = None\n        self.text_generator = None\n        self.available_backends = self._detect_backends()\n        self.setup_models()\n    \n    def _detect_backends(self) -> Dict[str, bool]:\n        \"\"\"Detect which LLM backends are available\"\"\"\n        backends = {\n            'sentence_transformers': False,\n            'transformers': False,\n            'ollama': False\n        }\n        \n        try:\n            import sentence_transformers\n            backends['sentence_transformers'] = True\n            print(\"✅ sentence-transformers available\")\n        except ImportError:\n            print(\"❌ sentence-transformers not available\")\n        \n        try:\n            import transformers\n            backends['transformers'] = True\n            print(\"✅ transformers available\")\n        except ImportError:\n            print(\"❌ transformers not available\")\n        \n        try:\n            import ollama\n            # Try to connect to Ollama\n            response = ollama.list()\n            backends['ollama'] = True\n            print(\"✅ Ollama available\")\n        except:\n            print(\"❌ Ollama not available (install Ollama and pull a model)\")\n        \n        return backends\n    \n    def setup_models(self):\n        \"\"\"Setup embedding and text generation models\"\"\"\n        print(\"\\n🔧 Setting up models...\")\n        \n        # Setup embedding model\n        if self.available_backends['sentence_transformers']:\n            try:\n                # Use a lightweight, fast model for embeddings\n                self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')\n                print(\"✅ Embedding model loaded: all-MiniLM-L6-v2\")\n            except Exception as e:\n                print(f\"⚠️ Could not load embedding model: {e}\")\n        \n        # Setup text generation model\n        self._setup_text_generator()\n    \n    def _setup_text_generator(self):\n        \"\"\"Setup text generation model with fallback options\"\"\"\n        \n        # Option 1: Try Ollama (best for local inference)\n        if self.available_backends['ollama']:\n            try:\n                import ollama\n                # Check if we have a suitable model\n                models = ollama.list().get('models', [])\n                suitable_models = ['llama3.1:8b', 'llama3.1', 'mistral', 'phi3']\n                \n                available_model = None\n                for model in suitable_models:\n                    if any(m['name'].startswith(model) for m in models):\n                        available_model = model\n                        break\n                \n                if available_model:\n                    self.text_generator = OllamaGenerator(available_model)\n                    print(f\"✅ Using Ollama model: {available_model}\")\n                    return\n                else:\n                    print(\"⚠️ No suitable Ollama models found\")\n                    print(\"💡 Install Ollama and run: ollama pull llama3.1:8b\")\n            except Exception as e:\n                print(f\"⚠️ Ollama setup failed: {e}\")\n        \n        # Option 2: HuggingFace Transformers (fallback)\n        if self.available_backends['transformers']:\n            try:\n                # Use a lightweight model for demonstration\n                self.text_generator = HuggingFaceGenerator('microsoft/DialoGPT-medium')\n                print(\"✅ Using HuggingFace model: microsoft/DialoGPT-medium\")\n                return\n            except Exception as e:\n                print(f\"⚠️ HuggingFace setup failed: {e}\")\n        \n        # Option 3: Simple rule-based fallback\n        self.text_generator = SimpleGenerator()\n        print(\"ℹ️ Using simple rule-based text generator (fallback)\")\n    \n    def get_embeddings(self, texts: List[str]) -> np.ndarray:\n        \"\"\"Get embeddings for a list of texts\"\"\"\n        if self.embedding_model is None:\n            # Fallback to simple TF-IDF if no model available\n            from sklearn.feature_extraction.text import TfidfVectorizer\n            vectorizer = TfidfVectorizer(max_features=384)  # Match embedding dimension\n            return vectorizer.fit_transform(texts).toarray()\n        \n        return self.embedding_model.encode(texts)\n    \n    def generate_text(self, prompt: str, max_length: int = 200) -> str:\n        \"\"\"Generate text using the available model\"\"\"\n        if self.text_generator is None:\n            return \"Text generation not available - please setup a model\"\n        \n        return self.text_generator.generate(prompt, max_length)\n\n# Different text generator implementations\nclass OllamaGenerator:\n    \"\"\"Ollama-based text generator\"\"\"\n    \n    def __init__(self, model_name: str):\n        import ollama\n        self.client = ollama\n        self.model_name = model_name\n    \n    def generate(self, prompt: str, max_length: int = 200) -> str:\n        try:\n            response = self.client.generate(\n                model=self.model_name,\n                prompt=prompt,\n                options={'num_predict': max_length}\n            )\n            return response.get('response', 'No response generated')\n        except Exception as e:\n            return f\"Error generating text: {e}\"\n\nclass HuggingFaceGenerator:\n    \"\"\"HuggingFace transformers-based generator\"\"\"\n    \n    def __init__(self, model_name: str):\n        self.generator = pipeline('text-generation', model=model_name)\n    \n    def generate(self, prompt: str, max_length: int = 200) -> str:\n        try:\n            outputs = self.generator(\n                prompt, \n                max_length=max_length, \n                num_return_sequences=1,\n                temperature=0.7\n            )\n            return outputs[0]['generated_text'].replace(prompt, '').strip()\n        except Exception as e:\n            return f\"Error generating text: {e}\"\n\nclass SimpleGenerator:\n    \"\"\"Simple rule-based generator for fallback\"\"\"\n    \n    def generate(self, prompt: str, max_length: int = 200) -> str:\n        # Simple template-based responses for financial analysis\n        if 'investment' in prompt.lower():\n            return \"Based on the available data, consider diversifying your portfolio and consulting with a financial advisor.\"\n        elif 'risk' in prompt.lower():\n            return \"Key risks include market volatility, regulatory changes, and competitive pressures. Monitor these factors closely.\"\n        elif 'recommend' in prompt.lower():\n            return \"Recommendations should be based on thorough analysis of financial statements, market conditions, and your investment goals.\"\n        else:\n            return \"Please provide more specific financial questions for detailed analysis.\"\n\n# Initialize the LLM manager\nprint(\"🚀 Initializing LLM Manager...\")\nllm_manager = LLMManager()\n\n# Test the setup\nprint(\"\\n🧪 Testing LLM Setup:\")\nif llm_manager.embedding_model:\n    test_texts = [\"Apple Inc. financial performance\", \"Technology sector risks\"]\n    embeddings = llm_manager.get_embeddings(test_texts)\n    print(f\"✅ Embeddings shape: {embeddings.shape}\")\nelse:\n    print(\"⚠️ No embedding model available\")\n\n# Test text generation\ntest_prompt = \"What are the key factors to consider when analyzing a technology company's financial health?\"\nresponse = llm_manager.generate_text(test_prompt, max_length=100)\nprint(f\"\\n🤖 Sample AI Response:\\n{response[:200]}...\")\n\nprint(\"\\n✅ LLM integration complete!\")"

### 🏗️ Financial Knowledge Graph Construction\n\nNow we'll build a knowledge graph from our financial data, including vector embeddings for semantic search:"

In [ ]:
# Financial Knowledge Graph Builder\nclass FinancialGraphBuilder:\n    \"\"\"Build and manage financial knowledge graph\"\"\"\n    \n    def __init__(self, neo4j_graph, llm_manager):\n        self.kg = neo4j_graph\n        self.llm = llm_manager\n        self.embedding_dimension = 384  # all-MiniLM-L6-v2 dimension\n    \n    def clear_financial_data(self):\n        \"\"\"Clear existing financial data (optional)\"\"\"\n        print(\"🧹 Clearing existing financial data...\")\n        cypher = \"\"\"\n        MATCH (n) \n        WHERE n:Company OR n:Document OR n:Section OR n:Risk OR n:Metric\n        DETACH DELETE n\n        \"\"\"\n        self.kg.query(cypher)\n        print(\"✅ Financial data cleared\")\n    \n    def create_vector_index(self):\n        \"\"\"Create vector index for semantic search\"\"\"\n        print(\"📊 Creating vector index for embeddings...\")\n        try:\n            # Create vector index for document sections\n            cypher = f\"\"\"\n            CREATE VECTOR INDEX section_embeddings IF NOT EXISTS\n            FOR (s:Section) ON (s.embedding)\n            OPTIONS {{\n                indexConfig: {{\n                    `vector.dimensions`: {self.embedding_dimension},\n                    `vector.similarity_function`: 'cosine'\n                }}\n            }}\n            \"\"\"\n            self.kg.query(cypher)\n            print(\"✅ Vector index created\")\n        except Exception as e:\n            print(f\"⚠️ Vector index creation: {e}\")\n    \n    def build_financial_graph(self, financial_documents: List[FinancialDocument]):\n        \"\"\"Build the complete financial knowledge graph\"\"\"\n        if not check_kg_available():\n            print(\"❌ Neo4j connection not available\")\n            return\n        \n        print(\"🏗️ Building Financial Knowledge Graph...\")\n        \n        # Create vector index first\n        self.create_vector_index()\n        \n        for doc in financial_documents:\n            print(f\"\\n📊 Processing {doc.company.name}...\")\n            \n            # 1. Create company node\n            company_id = self._create_company_node(doc.company)\n            \n            # 2. Create document node\n            doc_id = self._create_document_node(doc, company_id)\n            \n            # 3. Process each section\n            for section_name, content in doc.content.items():\n                self._create_section_node(section_name, content, doc_id, company_id)\n            \n            # 4. Extract and create risk nodes\n            if 'risk_factors' in doc.content:\n                self._extract_risks(doc.content['risk_factors'], company_id)\n            \n            # 5. Extract financial metrics (simplified)\n            self._extract_metrics(doc, company_id)\n        \n        print(\"\\n✅ Financial Knowledge Graph built successfully!\")\n        self._print_graph_stats()\n    \n    def _create_company_node(self, company: CompanyInfo) -> str:\n        \"\"\"Create company node\"\"\"\n        cypher = \"\"\"\n        MERGE (c:Company {ticker: $ticker})\n        SET c.name = $name,\n            c.cik = $cik,\n            c.sector = $sector,\n            c.industry = $industry,\n            c.created_at = datetime()\n        RETURN c.ticker AS ticker\n        \"\"\"\n        \n        result = self.kg.query(cypher, params={\n            'ticker': company.ticker,\n            'name': company.name,\n            'cik': company.cik,\n            'sector': company.sector,\n            'industry': company.industry\n        })\n        \n        return result[0]['ticker'] if result else company.ticker\n    \n    def _create_document_node(self, doc: FinancialDocument, company_id: str) -> str:\n        \"\"\"Create document node\"\"\"\n        doc_id = f\"{company_id}_{doc.document_type}_{doc.filing_date}\"\n        \n        cypher = \"\"\"\n        MATCH (c:Company {ticker: $company_id})\n        MERGE (d:Document {id: $doc_id})\n        SET d.type = $doc_type,\n            d.filing_date = date($filing_date),\n            d.period_end = date($period_end),\n            d.url = $url,\n            d.created_at = datetime()\n        MERGE (c)-[:FILED]->(d)\n        RETURN d.id AS doc_id\n        \"\"\"\n        \n        self.kg.query(cypher, params={\n            'company_id': company_id,\n            'doc_id': doc_id,\n            'doc_type': doc.document_type,\n            'filing_date': doc.filing_date,\n            'period_end': doc.period_end,\n            'url': doc.url\n        })\n        \n        return doc_id\n    \n    def _create_section_node(self, section_name: str, content: str, doc_id: str, company_id: str):\n        \"\"\"Create section node with embeddings\"\"\"\n        # Generate embedding for the content\n        try:\n            embedding = self.llm.get_embeddings([content])[0]\n            embedding_list = embedding.tolist()\n        except Exception as e:\n            print(f\"⚠️ Could not generate embedding for {section_name}: {e}\")\n            embedding_list = [0.0] * self.embedding_dimension\n        \n        section_id = f\"{doc_id}_{section_name}\"\n        \n        cypher = \"\"\"\n        MATCH (d:Document {id: $doc_id})\n        MATCH (c:Company {ticker: $company_id})\n        MERGE (s:Section {id: $section_id})\n        SET s.name = $section_name,\n            s.content = $content,\n            s.word_count = $word_count,\n            s.embedding = $embedding,\n            s.created_at = datetime()\n        MERGE (d)-[:CONTAINS]->(s)\n        MERGE (c)-[:HAS_SECTION]->(s)\n        \"\"\"\n        \n        self.kg.query(cypher, params={\n            'doc_id': doc_id,\n            'company_id': company_id,\n            'section_id': section_id,\n            'section_name': section_name.replace('_', ' ').title(),\n            'content': content[:5000],  # Limit content size for Neo4j\n            'word_count': len(content.split()),\n            'embedding': embedding_list\n        })\n    \n    def _extract_risks(self, risk_content: str, company_id: str):\n        \"\"\"Extract and create risk nodes from risk factors section\"\"\"\n        # Simple risk extraction (in production, use NER)\n        risk_keywords = [\n            'competition', 'regulatory', 'cybersecurity', 'market volatility',\n            'supply chain', 'economic conditions', 'technology changes',\n            'legal proceedings', 'intellectual property', 'data privacy'\n        ]\n        \n        content_lower = risk_content.lower()\n        \n        for risk_type in risk_keywords:\n            if risk_type in content_lower:\n                # Extract context around the risk\n                pattern = f'.{{0,100}}{re.escape(risk_type)}.{{0,100}}'\n                matches = re.findall(pattern, content_lower, re.IGNORECASE)\n                \n                if matches:\n                    risk_context = matches[0][:300]  # First match, limited length\n                    \n                    cypher = \"\"\"\n                    MATCH (c:Company {ticker: $company_id})\n                    MERGE (r:Risk {type: $risk_type, company: $company_id})\n                    SET r.description = $description,\n                        r.severity = $severity,\n                        r.created_at = datetime()\n                    MERGE (c)-[:FACES_RISK]->(r)\n                    \"\"\"\n                    \n                    # Simple severity assessment\n                    severity = 'high' if any(word in risk_context for word in ['critical', 'significant', 'major']) else 'medium'\n                    \n                    self.kg.query(cypher, params={\n                        'company_id': company_id,\n                        'risk_type': risk_type.title(),\n                        'description': risk_context,\n                        'severity': severity\n                    })\n    \n    def _extract_metrics(self, doc: FinancialDocument, company_id: str):\n        \"\"\"Extract financial metrics (simplified)\"\"\"\n        # Simple metric extraction for demo\n        metrics = {\n            'Revenue Growth': 'positive' if 'increased' in doc.content.get('management_discussion', '').lower() else 'unknown',\n            'Market Position': 'strong' if 'leading' in doc.content.get('business_overview', '').lower() else 'competitive'\n        }\n        \n        for metric_name, value in metrics.items():\n            cypher = \"\"\"\n            MATCH (c:Company {ticker: $company_id})\n            MERGE (m:Metric {name: $metric_name, company: $company_id})\n            SET m.value = $value,\n                m.period = $period,\n                m.created_at = datetime()\n            MERGE (c)-[:HAS_METRIC]->(m)\n            \"\"\"\n            \n            self.kg.query(cypher, params={\n                'company_id': company_id,\n                'metric_name': metric_name,\n                'value': value,\n                'period': doc.period_end\n            })\n    \n    def _print_graph_stats(self):\n        \"\"\"Print graph statistics\"\"\"\n        stats_queries = {\n            'Companies': \"MATCH (c:Company) RETURN count(c) AS count\",\n            'Documents': \"MATCH (d:Document) RETURN count(d) AS count\",\n            'Sections': \"MATCH (s:Section) RETURN count(s) AS count\",\n            'Risks': \"MATCH (r:Risk) RETURN count(r) AS count\",\n            'Metrics': \"MATCH (m:Metric) RETURN count(m) AS count\",\n        }\n        \n        print(\"\\n📈 Graph Statistics:\")\n        for entity, query in stats_queries.items():\n            result = self.kg.query(query)\n            count = result[0]['count'] if result else 0\n            print(f\"   {entity}: {count}\")\n\n# Build the financial knowledge graph\nif 'kg' in globals() and financial_documents:\n    print(\"🚀 Starting Financial Knowledge Graph construction...\")\n    \n    graph_builder = FinancialGraphBuilder(kg, llm_manager)\n    \n    # Optional: Clear existing financial data\n    # Uncomment the next line to start fresh\n    # graph_builder.clear_financial_data()\n    \n    # Build the graph\n    graph_builder.build_financial_graph(financial_documents)\nelse:\n    print(\"⚠️ Neo4j connection or financial documents not available\")\n    print(\"   Please ensure you've run the previous cells successfully\")"

### 💬 GraphRAG Investment Advisor\n\nNow we'll create an intelligent investment advisor that uses our knowledge graph and LLM to provide financial insights:"

In [ ]:
# GraphRAG Investment Advisor\nclass InvestmentAdvisor:\n    \"\"\"AI-powered investment advisor using GraphRAG\"\"\"\n    \n    def __init__(self, neo4j_graph, llm_manager):\n        self.kg = neo4j_graph\n        self.llm = llm_manager\n        self.embedding_dimension = 384\n    \n    def semantic_search(self, query: str, limit: int = 5) -> List[Dict]:\n        \"\"\"Semantic search using vector embeddings\"\"\"\n        try:\n            # Get query embedding\n            query_embedding = self.llm.get_embeddings([query])[0].tolist()\n            \n            # Search for similar sections\n            cypher = \"\"\"\n            CALL db.index.vector.queryNodes('section_embeddings', $limit, $query_embedding) \n            YIELD node AS section, score\n            MATCH (section)<-[:HAS_SECTION]-(company:Company)\n            RETURN \n                company.name AS company,\n                company.ticker AS ticker,\n                section.name AS section_type,\n                section.content AS content,\n                score,\n                section.word_count AS word_count\n            ORDER BY score DESC\n            \"\"\"\n            \n            result = self.kg.query(cypher, params={\n                'query_embedding': query_embedding,\n                'limit': limit\n            })\n            \n            return result\n            \n        except Exception as e:\n            print(f\"⚠️ Semantic search error: {e}\")\n            return self._fallback_search(query, limit)\n    \n    def _fallback_search(self, query: str, limit: int = 5) -> List[Dict]:\n        \"\"\"Fallback text search when vector search isn't available\"\"\"\n        query_lower = query.lower()\n        \n        cypher = \"\"\"\n        MATCH (c:Company)-[:HAS_SECTION]->(s:Section)\n        WHERE toLower(s.content) CONTAINS $query\n           OR toLower(s.name) CONTAINS $query\n           OR toLower(c.name) CONTAINS $query\n        RETURN \n            c.name AS company,\n            c.ticker AS ticker,\n            s.name AS section_type,\n            s.content AS content,\n            0.8 AS score,  // Dummy score for fallback\n            s.word_count AS word_count\n        LIMIT $limit\n        \"\"\"\n        \n        return self.kg.query(cypher, params={'query': query_lower, 'limit': limit})\n    \n    def get_company_overview(self, ticker: str) -> Dict:\n        \"\"\"Get comprehensive company overview\"\"\"\n        cypher = \"\"\"\n        MATCH (c:Company {ticker: $ticker})\n        OPTIONAL MATCH (c)-[:FACES_RISK]->(r:Risk)\n        OPTIONAL MATCH (c)-[:HAS_METRIC]->(m:Metric)\n        OPTIONAL MATCH (c)-[:HAS_SECTION]->(s:Section)\n        RETURN \n            c.name AS company_name,\n            c.sector AS sector,\n            c.industry AS industry,\n            COLLECT(DISTINCT r.type) AS risks,\n            COLLECT(DISTINCT {name: m.name, value: m.value}) AS metrics,\n            COLLECT(DISTINCT s.name) AS available_sections\n        \"\"\"\n        \n        result = self.kg.query(cypher, params={'ticker': ticker.upper()})\n        return result[0] if result else {}\n    \n    def analyze_risks(self, ticker: str = None) -> List[Dict]:\n        \"\"\"Analyze risks for a company or across all companies\"\"\"\n        if ticker:\n            cypher = \"\"\"\n            MATCH (c:Company {ticker: $ticker})-[:FACES_RISK]->(r:Risk)\n            RETURN \n                c.name AS company,\n                r.type AS risk_type,\n                r.severity AS severity,\n                r.description AS description\n            ORDER BY \n                CASE r.severity WHEN 'high' THEN 1 WHEN 'medium' THEN 2 ELSE 3 END,\n                r.type\n            \"\"\"\n            params = {'ticker': ticker.upper()}\n        else:\n            cypher = \"\"\"\n            MATCH (c:Company)-[:FACES_RISK]->(r:Risk)\n            RETURN \n                c.name AS company,\n                r.type AS risk_type,\n                r.severity AS severity,\n                r.description AS description\n            ORDER BY \n                CASE r.severity WHEN 'high' THEN 1 WHEN 'medium' THEN 2 ELSE 3 END,\n                c.name\n            \"\"\"\n            params = {}\n        \n        return self.kg.query(cypher, params=params)\n    \n    def compare_companies(self, tickers: List[str]) -> Dict:\n        \"\"\"Compare multiple companies across key metrics\"\"\"\n        ticker_list = [t.upper() for t in tickers]\n        \n        cypher = \"\"\"\n        MATCH (c:Company)\n        WHERE c.ticker IN $tickers\n        OPTIONAL MATCH (c)-[:FACES_RISK]->(r:Risk)\n        OPTIONAL MATCH (c)-[:HAS_METRIC]->(m:Metric)\n        RETURN \n            c.ticker AS ticker,\n            c.name AS company_name,\n            c.sector AS sector,\n            COUNT(DISTINCT r) AS total_risks,\n            COUNT(DISTINCT CASE WHEN r.severity = 'high' THEN r END) AS high_risks,\n            COLLECT(DISTINCT r.type) AS risk_types,\n            COLLECT(DISTINCT {name: m.name, value: m.value}) AS metrics\n        ORDER BY c.ticker\n        \"\"\"\n        \n        results = self.kg.query(cypher, params={'tickers': ticker_list})\n        \n        # Structure the comparison\n        comparison = {\n            'companies': results,\n            'summary': self._generate_comparison_summary(results)\n        }\n        \n        return comparison\n    \n    def _generate_comparison_summary(self, company_data: List[Dict]) -> str:\n        \"\"\"Generate a summary of company comparison\"\"\"\n        if not company_data:\n            return \"No companies found for comparison.\"\n        \n        # Simple rule-based summary\n        lowest_risk = min(company_data, key=lambda x: x['total_risks'])\n        highest_risk = max(company_data, key=lambda x: x['total_risks'])\n        \n        summary = f\"\"\"\n        Company Comparison Summary:\n        • Lowest Risk: {lowest_risk['company_name']} ({lowest_risk['total_risks']} risks)\n        • Highest Risk: {highest_risk['company_name']} ({highest_risk['total_risks']} risks)\n        • All companies operate in similar sectors: {', '.join(set(c['sector'] for c in company_data))}\n        \"\"\"\n        \n        return summary.strip()\n    \n    def ask_advisor(self, question: str) -> str:\n        \"\"\"Main advisor interface - answer investment questions using GraphRAG\"\"\"\n        print(f\"🤔 Question: {question}\")\n        print(\"🔍 Searching knowledge graph...\")\n        \n        # Perform semantic search\n        relevant_sections = self.semantic_search(question, limit=3)\n        \n        if not relevant_sections:\n            return \"I couldn't find relevant information in the financial database. Please try a more specific question about the companies in our database.\"\n        \n        # Build context from search results\n        context = \"\\n\\n\".join([\n            f\"**{section['company']} ({section['ticker']}) - {section['section_type']}:**\\n{section['content'][:500]}...\"\n            for section in relevant_sections\n        ])\n        \n        # Generate response using LLM with context\n        prompt = f\"\"\"\n        You are a financial investment advisor. Based on the following financial information from SEC filings, \n        please provide a helpful and informative response to the investor's question.\n        \n        Question: {question}\n        \n        Relevant Financial Information:\n        {context}\n        \n        Please provide:\n        1. A direct answer to the question\n        2. Key insights from the financial data\n        3. Important considerations for investors\n        4. Any relevant risks or opportunities\n        \n        Response:\n        \"\"\"\n        \n        response = self.llm.generate_text(prompt, max_length=300)\n        \n        return response\n    \n    def investment_screening(self, criteria: Dict) -> List[Dict]:\n        \"\"\"Screen companies based on investment criteria\"\"\"\n        # Build dynamic query based on criteria\n        where_conditions = []\n        params = {}\n        \n        if 'sector' in criteria:\n            where_conditions.append(\"c.sector = $sector\")\n            params['sector'] = criteria['sector']\n        \n        if 'max_risks' in criteria:\n            where_conditions.append(\"riskCount <= $max_risks\")\n            params['max_risks'] = criteria['max_risks']\n        \n        where_clause = \" AND \".join(where_conditions) if where_conditions else \"true\"\n        \n        cypher = f\"\"\"\n        MATCH (c:Company)\n        OPTIONAL MATCH (c)-[:FACES_RISK]->(r:Risk)\n        WITH c, COUNT(r) AS riskCount\n        WHERE {where_clause}\n        RETURN \n            c.ticker AS ticker,\n            c.name AS company_name,\n            c.sector AS sector,\n            c.industry AS industry,\n            riskCount AS total_risks\n        ORDER BY riskCount ASC, c.ticker\n        \"\"\"\n        \n        return self.kg.query(cypher, params=params)\n\n# Initialize the Investment Advisor\nif 'kg' in globals() and 'llm_manager' in globals():\n    advisor = InvestmentAdvisor(kg, llm_manager)\n    print(\"🎯 Investment Advisor initialized successfully!\")\n    \n    # Show available companies\n    print(\"\\n📋 Available companies for analysis:\")\n    companies = advisor.kg.query(\"MATCH (c:Company) RETURN c.ticker AS ticker, c.name AS name ORDER BY c.ticker\")\n    for company in companies:\n        print(f\"   {company['ticker']}: {company['name']}\")\nelse:\n    print(\"⚠️ Cannot initialize advisor - missing dependencies\")"

### 🎪 Demo: Financial Investment Advisor in Action\n\nLet's test our GraphRAG investment advisor with real financial questions:"

In [ ]:
# Demo: Investment Advisor in Action\nif 'advisor' in globals():\n    print(\"🎪 GraphRAG Financial Investment Advisor Demo\")\n    print(\"=\" * 60)\n    \n    # Demo 1: Company Overview\n    print(\"\\n1️⃣ Company Overview Analysis\")\n    print(\"-\" * 35)\n    overview = advisor.get_company_overview('AAPL')\n    if overview:\n        print(f\"Company: {overview.get('company_name', 'N/A')}\")\n        print(f\"Sector: {overview.get('sector', 'N/A')}\")\n        print(f\"Industry: {overview.get('industry', 'N/A')}\")\n        print(f\"Identified Risks: {', '.join(overview.get('risks', []))}\")\n        print(f\"Available Sections: {', '.join(overview.get('available_sections', []))}\")\n    else:\n        print(\"No company data found\")\n    \n    # Demo 2: Risk Analysis\n    print(\"\\n2️⃣ Risk Analysis\")\n    print(\"-\" * 20)\n    risks = advisor.analyze_risks('AAPL')\n    if risks:\n        for risk in risks[:3]:  # Show first 3 risks\n            print(f\"🚨 {risk['risk_type']} ({risk['severity']} severity)\")\n            print(f\"   {risk['description'][:100]}...\")\n    else:\n        print(\"No risks found\")\n    \n    # Demo 3: Company Comparison\n    print(\"\\n3️⃣ Company Comparison\")\n    print(\"-\" * 25)\n    comparison = advisor.compare_companies(['AAPL', 'MSFT'])\n    if comparison['companies']:\n        print(\"📊 Comparison Results:\")\n        for company in comparison['companies']:\n            print(f\"   {company['ticker']}: {company['total_risks']} risks, {company['high_risks']} high-severity\")\n        print(f\"\\n💡 {comparison['summary']}\")\n    \n    # Demo 4: Investment Screening\n    print(\"\\n4️⃣ Investment Screening\")\n    print(\"-\" * 25)\n    criteria = {'sector': 'Technology', 'max_risks': 5}\n    screened = advisor.investment_screening(criteria)\n    if screened:\n        print(f\"Companies matching criteria (Technology sector, ≤5 risks):\")\n        for company in screened:\n            print(f\"   ✅ {company['ticker']}: {company['company_name']} ({company['total_risks']} risks)\")\n    \n    # Demo 5: Ask the Advisor (Main GraphRAG feature)\n    print(\"\\n5️⃣ AI Investment Advisor (GraphRAG)\")\n    print(\"-\" * 40)\n    \n    # Example questions\n    questions = [\n        \"What are the main risks facing Apple?\",\n        \"Should I invest in technology companies?\",\n        \"How is Microsoft's business performing?\"\n    ]\n    \n    for question in questions[:2]:  # Demo first 2 questions\n        print(f\"\\n❓ Question: {question}\")\n        print(\"🤖 AI Response:\")\n        response = advisor.ask_advisor(question)\n        print(f\"   {response[:300]}...\")  # Show first 300 characters\n        print()\n    \n    print(\"\\n✨ Try asking your own questions about the companies in our database!\")\n    print(\"   Available companies: AAPL, MSFT (expand with more SEC data)\")\n    \nelse:\n    print(\"❌ Investment Advisor not available\")\n    print(\"Please run the previous cells to initialize the advisor\")"

### 🎯 Section Summary and Next Steps\n\n**🎉 Congratulations!** You've built a complete GraphRAG Financial Investment Advisor using open-source technologies!\n\n#### ✅ **What You've Accomplished:**\n- **📊 SEC Data Integration**: Real financial data extraction and processing\n- **🤖 Open Source AI**: LLM integration with Ollama/HuggingFace fallbacks\n- **🏗️ Knowledge Graph**: Financial entities, relationships, and vector embeddings\n- **💬 GraphRAG System**: Semantic search + AI-generated insights\n- **🎯 Business Application**: Practical investment advisor with real value\n\n#### 🚀 **Production Enhancements:**\n1. **Real SEC API Integration**: Replace demo data with live SEC EDGAR feeds\n2. **Advanced NER**: Use spaCy or custom models for better entity extraction\n3. **Financial Metrics**: Integrate quantitative financial ratios and calculations\n4. **News Integration**: Add real-time financial news and sentiment analysis\n5. **User Interface**: Build a web app with Streamlit or FastAPI\n6. **Alerts System**: Set up notifications for risk changes or opportunities\n\n#### 🔧 **Technical Improvements:**\n1. **Scaling**: Implement data pipelines for larger datasets\n2. **Performance**: Optimize vector searches and caching\n3. **Security**: Add user authentication and data privacy controls\n4. **Monitoring**: Add logging, metrics, and error tracking\n\n#### 💡 **Business Extensions:**\n1. **Portfolio Management**: Track and analyze investment portfolios\n2. **Risk Modeling**: Quantitative risk assessment and VaR calculations\n3. **ESG Analysis**: Environmental, social, and governance scoring\n4. **Regulatory Compliance**: Automated compliance checking and reporting\n\n---\n\n**🎊 You now have a working GraphRAG system that demonstrates the power of combining Knowledge Graphs with AI for real-world business applications!**"

## 12. Index Management and Query Profiling\n\n### 📇 Understanding Neo4j Indexes\n\nIndexes in Neo4j dramatically improve query performance by creating fast lookup structures for node properties and relationships.\n\n#### 🔍 Types of Indexes:\n- **Range Index**: For exact matches and range queries (numbers, dates, strings)\n- **Text Index**: For full-text search capabilities\n- **Point Index**: For spatial/geographic data\n- **Composite Index**: Combines multiple properties\n\n#### 📊 When to Create Indexes:\n- Properties used frequently in `MATCH` clauses\n- Properties in `WHERE` conditions\n- Properties used for `ORDER BY`\n- Unique constraints (automatically create indexes)\n\nLet's demonstrate index creation and performance analysis:"

In [ ]:
# Index Management Demonstration\nif not check_kg_available():\n    print(\"⏸️ Skipping index demo - connection not available\")\nelse:\n    print(\"📇 Index Management and Performance Analysis\")\n    print(\"=\" * 50)\n    \n    # Step 1: Check existing indexes\n    print(\"\\n1️⃣ Current Database Indexes:\")\n    try:\n        # Try modern SHOW INDEXES command\n        result = kg.query(\"SHOW INDEXES\")\n        if result:\n            print(f\"   Found {len(result)} existing indexes:\")\n            for idx in result[:5]:  # Show first 5\n                name = idx.get('name', 'unnamed')\n                labels = idx.get('labelsOrTypes', [])\n                properties = idx.get('properties', [])\n                state = idx.get('state', 'unknown')\n                print(f\"   📋 {name}: {labels} - {properties} ({state})\")\n        else:\n            print(\"   ℹ️ No indexes found\")\n    except Exception as e:\n        # Fallback for older Neo4j versions\n        print(f\"   ⚠️ Could not retrieve indexes: {str(e)[:50]}...\")\n        print(\"   (This may be due to Neo4j version or permissions)\")\n    \n    # Step 2: Create useful indexes for our movie data\n    print(\"\\n2️⃣ Creating Performance Indexes:\")\n    \n    indexes_to_create = [\n        {\n            \"name\": \"person_name_index\",\n            \"command\": \"CREATE INDEX person_name_index IF NOT EXISTS FOR (p:Person) ON (p.name)\",\n            \"description\": \"Index on Person.name for faster actor/director lookups\"\n        },\n        {\n            \"name\": \"movie_title_index\", \n            \"command\": \"CREATE INDEX movie_title_index IF NOT EXISTS FOR (m:Movie) ON (m.title)\",\n            \"description\": \"Index on Movie.title for faster movie searches\"\n        },\n        {\n            \"name\": \"movie_year_index\",\n            \"command\": \"CREATE INDEX movie_year_index IF NOT EXISTS FOR (m:Movie) ON (m.released)\",\n            \"description\": \"Index on Movie.released for year-based queries\"\n        },\n        {\n            \"name\": \"genre_name_index\",\n            \"command\": \"CREATE INDEX genre_name_index IF NOT EXISTS FOR (g:Genre) ON (g.name)\", \n            \"description\": \"Index on Genre.name for genre-based filtering\"\n        }\n    ]\n    \n    for idx in indexes_to_create:\n        try:\n            kg.query(idx[\"command\"])\n            print(f\"   ✅ {idx['name']}: {idx['description']}\")\n        except Exception as e:\n            print(f\"   ⚠️ {idx['name']}: {str(e)[:60]}...\")\n    \n    print(\"\\n   💡 Indexes may take a moment to build in the background\")"

### 📊 Query Profiling with EXPLAIN and PROFILE\n\n**EXPLAIN** shows the query execution plan without running the query, while **PROFILE** executes the query and shows actual performance metrics.\n\n#### Key Metrics to Watch:\n- **db hits**: Number of operations against the graph store\n- **rows**: Number of rows processed at each step\n- **time**: Actual execution time (PROFILE only)\n- **memory**: Memory usage (PROFILE only)\n- **Index usage**: Whether indexes are being utilized\n\nLet's compare query performance with and without indexes:"

In [ ]:
# Query Profiling and Performance Analysis\nif not check_kg_available():\n    print(\"⏸️ Skipping profiling demo - connection not available\")\nelse:\n    print(\"📊 Query Profiling and Performance Analysis\")\n    print(\"=\" * 50)\n    \n    # Helper function to format profiling results\n    def format_profile_info(result):\n        if not result or len(result) == 0:\n            return \"No profile data available\"\n        \n        # Extract key metrics from profile\n        info = []\n        for row in result:\n            for key, value in row.items():\n                if isinstance(value, str) and ('db hits' in value.lower() or 'rows' in value.lower()):\n                    info.append(value[:100])  # Truncate long strings\n        return \"\\n   \".join(info[:3]) if info else \"Profile data format not recognized\"\n    \n    # Demo 1: EXPLAIN - Show execution plan without running\n    print(\"\\n1️⃣ EXPLAIN - Query Execution Plan (without execution):\")\n    print(\"   Query: Find Tom Hanks movies\")\n    \n    try:\n        explain_result = kg.query(\"\"\"\n        EXPLAIN \n        MATCH (p:Person {name: \"Tom Hanks\"})-[:ACTED_IN]->(m:Movie)\n        RETURN p.name, m.title, m.released\n        \"\"\")\n        \n        print(\"   📋 Execution Plan:\")\n        if explain_result:\n            for i, step in enumerate(explain_result[:3]):  # Show first 3 steps\n                print(f\"   Step {i+1}: {list(step.keys())[0] if step else 'Unknown'}\")\n        else:\n            print(\"   ⚠️ No execution plan data returned\")\n            \n    except Exception as e:\n        print(f\"   ⚠️ EXPLAIN failed: {str(e)[:60]}...\")\n    \n    # Demo 2: PROFILE - Show actual performance metrics\n    print(\"\\n2️⃣ PROFILE - Actual Performance Metrics:\")\n    print(\"   Query: Find all Crime movies with their actors\")\n    \n    import time\n    start_time = time.time()\n    \n    try:\n        profile_result = kg.query(\"\"\"\n        PROFILE\n        MATCH (m:Movie)-[:BELONGS_TO]->(g:Genre {name: \"Crime\"})\n        MATCH (m)<-[:ACTED_IN]-(a:Person)\n        RETURN m.title AS movie, m.released AS year, collect(a.name)[0..3] AS actors\n        ORDER BY m.released DESC\n        \"\"\")\n        \n        execution_time = time.time() - start_time\n        \n        print(f\"   ⏱️ Query executed in {execution_time:.4f} seconds\")\n        print(f\"   📊 Returned {len(profile_result)} rows\")\n        \n        # Show sample results\n        print(\"\\n   📋 Sample Results:\")\n        for row in profile_result[:2]:  # Show first 2 movies\n            actors_str = \", \".join(row['actors'][:3])\n            print(f\"   🎬 {row['movie']} ({row['year']}) - {actors_str}...\")\n            \n        print(\"\\n   📈 Performance Profile Available (detailed metrics in Neo4j Browser)\")\n        \n    except Exception as e:\n        print(f\"   ⚠️ PROFILE failed: {str(e)[:60]}...\")\n    \n    # Demo 3: Compare indexed vs non-indexed performance\n    print(\"\\n3️⃣ Performance Comparison - Index Usage:\")\n    \n    # Query that should use Person.name index\n    queries_to_compare = [\n        {\n            \"name\": \"Indexed Property Search\",\n            \"query\": \"MATCH (p:Person {name: 'Leonardo DiCaprio'}) RETURN p.name, p.born\",\n            \"explanation\": \"Uses index on Person.name (should be fast)\"\n        },\n        {\n            \"name\": \"Range Query on Indexed Property\", \n            \"query\": \"MATCH (m:Movie) WHERE m.released >= 2000 AND m.released <= 2010 RETURN m.title, m.released ORDER BY m.released\",\n            \"explanation\": \"Uses index on Movie.released for range query\"\n        },\n        {\n            \"name\": \"Non-indexed Property Filter\",\n            \"query\": \"MATCH (m:Movie) WHERE m.tagline CONTAINS 'you' RETURN m.title, m.tagline LIMIT 3\",\n            \"explanation\": \"No index on tagline - requires full scan\"\n        }\n    ]\n    \n    for i, query_info in enumerate(queries_to_compare):\n        print(f\"\\n   Query {i+1}: {query_info['name']}\")\n        print(f\"   💡 {query_info['explanation']}\")\n        \n        start_time = time.time()\n        try:\n            result = kg.query(query_info['query'])\n            execution_time = time.time() - start_time\n            \n            print(f\"   ⏱️ Executed in {execution_time:.4f} seconds ({len(result)} rows)\")\n            \n            # Show first result if available\n            if result:\n                first_result = result[0]\n                result_str = \", \".join([f\"{k}: {v}\" for k, v in first_result.items()])\n                print(f\"   📋 Sample: {result_str[:80]}...\")\n                \n        except Exception as e:\n            print(f\"   ❌ Query failed: {str(e)[:50]}...\")"

### 🔬 Advanced Profiling Techniques\n\nFor deeper performance analysis, Neo4j provides additional profiling capabilities:"

In [ ]:
# Advanced Profiling Techniques\nif not check_kg_available():\n    print(\"⏸️ Skipping advanced profiling - connection not available\")\nelse:\n    print(\"🔬 Advanced Profiling Techniques\")\n    print(\"=\" * 40)\n    \n    # Demo 1: Memory profiling for large result sets\n    print(\"\\n1️⃣ Memory Usage Analysis:\")\n    print(\"   Comparing memory-efficient vs memory-intensive queries\")\n    \n    # Memory-efficient query (using aggregation)\n    print(\"\\n   Memory-Efficient Query (Aggregation):\")\n    start_time = time.time()\n    try:\n        result1 = kg.query(\"\"\"\n        MATCH (g:Genre)<-[:BELONGS_TO]-(m:Movie)\n        RETURN g.name AS genre, count(m) AS movieCount\n        ORDER BY movieCount DESC\n        \"\"\")\n        time1 = time.time() - start_time\n        print(f\"   ⏱️ Aggregation query: {time1:.4f} seconds, {len(result1)} rows\")\n        print(f\"   💾 Low memory usage (only final aggregated results)\")\n        \n    except Exception as e:\n        print(f\"   ❌ Query failed: {str(e)[:50]}...\")\n    \n    # Memory-intensive query (collecting all items)\n    print(\"\\n   Memory-Intensive Query (Collection):\")\n    start_time = time.time()\n    try:\n        result2 = kg.query(\"\"\"\n        MATCH (g:Genre)<-[:BELONGS_TO]-(m:Movie)\n        RETURN g.name AS genre, collect(m.title) AS allMovies\n        ORDER BY g.name\n        \"\"\")\n        time2 = time.time() - start_time\n        print(f\"   ⏱️ Collection query: {time2:.4f} seconds, {len(result2)} rows\")\n        print(f\"   💾 Higher memory usage (stores all movie titles in memory)\")\n        \n        # Show memory impact\n        if result2:\n            total_movies = sum(len(row['allMovies']) for row in result2)\n            print(f\"   📊 Total items in memory: {total_movies} movie titles\")\n            \n    except Exception as e:\n        print(f\"   ❌ Query failed: {str(e)[:50]}...\")\n    \n    # Demo 2: Index hit analysis\n    print(\"\\n2️⃣ Index Effectiveness Analysis:\")\n    \n    # Function to simulate index usage analysis\n    def analyze_query_performance(query_name, query, expected_index_usage=True):\n        print(f\"\\n   🔍 {query_name}:\")\n        start_time = time.time()\n        try:\n            result = kg.query(query)\n            execution_time = time.time() - start_time\n            \n            rows_returned = len(result) if result else 0\n            print(f\"   ⏱️ Time: {execution_time:.4f}s, Rows: {rows_returned}\")\n            \n            if expected_index_usage:\n                if execution_time < 0.01:  # Very fast suggests index usage\n                    print(f\"   ✅ Likely using index (very fast execution)\")\n                elif execution_time < 0.05:\n                    print(f\"   ⚡ Possibly using index (fast execution)\")\n                else:\n                    print(f\"   ⚠️ May not be using index (slower execution)\")\n            else:\n                print(f\"   📊 Full scan expected (no suitable index)\")\n                \n        except Exception as e:\n            print(f\"   ❌ Query failed: {str(e)[:50]}...\")\n    \n    # Test different query patterns\n    analyze_query_performance(\n        \"Exact Match (Should Use Index)\",\n        \"MATCH (p:Person {name: 'Tom Hanks'}) RETURN p\",\n        expected_index_usage=True\n    )\n    \n    analyze_query_performance(\n        \"Range Query (Should Use Index)\", \n        \"MATCH (m:Movie) WHERE m.released >= 2000 RETURN m.title, m.released LIMIT 5\",\n        expected_index_usage=True\n    )\n    \n    analyze_query_performance(\n        \"Pattern Match (May Not Use Index)\",\n        \"MATCH (p:Person) WHERE p.name STARTS WITH 'Tom' RETURN p.name LIMIT 5\",\n        expected_index_usage=False\n    )\n    \n    analyze_query_performance(\n        \"Full Property Scan (No Index)\",\n        \"MATCH (m:Movie) WHERE m.tagline CONTAINS 'the' RETURN m.title, m.tagline LIMIT 3\",\n        expected_index_usage=False\n    )\n    \n    # Demo 3: Query optimization suggestions\n    print(\"\\n3️⃣ Query Optimization Tips:\")\n    optimization_tips = [\n        \"✅ Use specific labels in MATCH clauses: MATCH (p:Person) not MATCH (p)\",\n        \"✅ Start with most selective patterns: specific nodes before broad patterns\",\n        \"✅ Use indexed properties for filtering: WHERE p.name = 'value'\",\n        \"✅ Apply LIMIT early: add LIMIT before expensive operations\",\n        \"✅ Use DISTINCT sparingly: only when absolutely necessary\",\n        \"✅ Prefer aggregation over collection: count() instead of collect()\",\n        \"✅ Use OPTIONAL MATCH for optional relationships\",\n        \"✅ Create composite indexes for multi-property queries\"\n    ]\n    \n    for tip in optimization_tips:\n        print(f\"   {tip}\")\n    \n    print(\"\\n💡 Pro Tip: Use Neo4j Browser for visual query profiling!\")\n    print(\"   • Open Neo4j Browser at your database URL\")\n    print(\"   • Prefix queries with PROFILE or EXPLAIN\")\n    print(\"   • View execution plans and performance metrics graphically\")"

### 🧹 Index Maintenance and Cleanup\n\nProper index management includes monitoring and cleanup:"

In [ ]:
# Index Maintenance and Management\nif not check_kg_available():\n    print(\"⏸️ Skipping index maintenance demo - connection not available\")\nelse:\n    print(\"🧹 Index Maintenance and Management\")\n    print(\"=\" * 40)\n    \n    # Demo 1: Monitor index usage and health\n    print(\"\\n1️⃣ Index Health Check:\")\n    \n    try:\n        # Check index status\n        result = kg.query(\"SHOW INDEXES\")\n        if result:\n            print(f\"   📊 Found {len(result)} indexes:\")\n            \n            # Categorize indexes by state\n            index_states = {}\n            for idx in result:\n                state = idx.get('state', 'unknown')\n                index_states[state] = index_states.get(state, 0) + 1\n            \n            for state, count in index_states.items():\n                status_emoji = \"✅\" if state.lower() == \"online\" else \"⚠️\"\n                print(f\"   {status_emoji} {state}: {count} indexes\")\n                \n            # Show any failed indexes\n            failed_indexes = [idx for idx in result if idx.get('state', '').lower() == 'failed']\n            if failed_indexes:\n                print(\"\\n   ❌ Failed indexes found:\")\n                for idx in failed_indexes:\n                    print(f\"     - {idx.get('name', 'unnamed')}: {idx.get('failureMessage', 'Unknown error')}\")\n            \n        else:\n            print(\"   ℹ️ No indexes found or unable to retrieve index information\")\n            \n    except Exception as e:\n        print(f\"   ⚠️ Could not check index health: {str(e)[:60]}...\")\n    \n    # Demo 2: Example of dropping unused indexes (commented for safety)\n    print(\"\\n2️⃣ Index Cleanup (Educational Example):\")\n    print(\"   💡 In production, regularly review and remove unused indexes\")\n    \n    cleanup_examples = [\n        {\n            \"scenario\": \"Remove index on rarely queried property\",\n            \"command\": \"DROP INDEX movie_tagline_index IF EXISTS\",\n            \"reason\": \"Tagline is rarely used in WHERE clauses\"\n        },\n        {\n            \"scenario\": \"Drop duplicate indexes\", \n            \"command\": \"DROP INDEX old_person_name_index IF EXISTS\",\n            \"reason\": \"Replaced by composite index on (name, born)\"\n        },\n        {\n            \"scenario\": \"Remove index on changed schema\",\n            \"command\": \"DROP INDEX deprecated_property_index IF EXISTS\",\n            \"reason\": \"Property no longer exists in data model\"\n        }\n    ]\n    \n    for example in cleanup_examples:\n        print(f\"\\n   🗑️ {example['scenario']}:\")\n        print(f\"      Command: {example['command']}\")\n        print(f\"      Reason: {example['reason']}\")\n    \n    print(\"\\n   ⚠️ Note: Above commands are examples only - not executed for safety\")\n    \n    # Demo 3: Index performance recommendations\n    print(\"\\n3️⃣ Index Performance Recommendations:\")\n    \n    recommendations = [\n        {\n            \"category\": \"🎯 Essential Indexes\",\n            \"items\": [\n                \"Create indexes on frequently filtered properties (name, id, status)\",\n                \"Index properties used in JOIN-like operations (relationship endpoints)\",\n                \"Index properties used in ORDER BY clauses\"\n            ]\n        },\n        {\n            \"category\": \"⚡ Performance Indexes\", \n            \"items\": [\n                \"Composite indexes for multi-property queries\",\n                \"Range indexes for date/numeric range queries\",\n                \"Text indexes for full-text search requirements\"\n            ]\n        },\n        {\n            \"category\": \"🚫 Avoid Over-Indexing\",\n            \"items\": [\n                \"Don't index properties that are never queried\",\n                \"Avoid indexes on highly variable properties (timestamps, UUIDs)\",\n                \"Remove indexes on properties that are always used with other indexed properties\"\n            ]\n        },\n        {\n            \"category\": \"📊 Monitoring\",\n            \"items\": [\n                \"Regularly check index usage statistics\",\n                \"Monitor query performance after index changes\", \n                \"Review slow query logs to identify missing indexes\"\n            ]\n        }\n    ]\n    \n    for rec in recommendations:\n        print(f\"\\n   {rec['category']}:\")\n        for item in rec['items']:\n            print(f\"     • {item}\")\n    \n    # Demo 4: Composite index example\n    print(\"\\n4️⃣ Advanced: Composite Index Example:\")\n    print(\"   💡 For queries that filter on multiple properties\")\n    \n    composite_example = \"\"\"\n    -- Example: Queries that filter by both person name and birth year\n    MATCH (p:Person) \n    WHERE p.name = 'Tom Hanks' AND p.born = 1956 \n    RETURN p\n    \n    -- Optimized with composite index:\n    CREATE INDEX person_name_born_index IF NOT EXISTS \n    FOR (p:Person) ON (p.name, p.born)\n    \"\"\"\n    \n    print(f\"   📝 Use Case:{composite_example}\")\n    print(\"   ⚡ Benefits: Single index lookup instead of multiple index intersections\")\n    print(\"   📏 Trade-off: Larger index size, useful only for queries using both properties\")\n    \n    print(\"\\n🎓 Key Takeaways:\")\n    takeaways = [\n        \"Indexes dramatically improve query performance but use storage space\",\n        \"Create indexes on properties used in MATCH, WHERE, and ORDER BY clauses\",\n        \"Use EXPLAIN/PROFILE to verify index usage in your queries\",\n        \"Regular maintenance: monitor, review, and cleanup unused indexes\",\n        \"Balance performance gains vs. storage/maintenance costs\"\n    ]\n    \n    for takeaway in takeaways:\n        print(f\"   ✅ {takeaway}\")"

## Appendix: Quick Reference

### 🔖 Cypher Quick Reference

| Pattern | Description | Example |
|---------|-------------|--------|
| `(n)` | Any node | `MATCH (n) RETURN n` |
| `(n:Label)` | Node with label | `MATCH (p:Person) RETURN p` |
| `(n {prop: "value"})` | Node with property | `MATCH (p {name: "Alice"}) RETURN p` |
| `-[:TYPE]->` | Directed relationship | `MATCH (a)-[:KNOWS]->(b) RETURN a, b` |
| `-[:TYPE*1..3]-` | Variable length path | `MATCH (a)-[:KNOWS*1..3]-(b) RETURN a, b` |
| `OPTIONAL MATCH` | Optional pattern | `OPTIONAL MATCH (p)-[:ACTED_IN]->(m)` |
| `WHERE` | Filter condition | `WHERE p.age > 30` |
| `WITH` | Pass data between clauses | `WITH p, count(*) AS connections` |
| `ORDER BY` | Sort results | `ORDER BY p.name ASC` |
| `LIMIT` | Restrict count | `LIMIT 10` |
| `COLLECT()` | Aggregate to list | `COLLECT(p.name)` |
| `COUNT()` | Count items | `COUNT(p)` |
| `CASE WHEN` | Conditional logic | `CASE WHEN p.age > 30 THEN "Adult"` |

### 📁 Data Files Structure

The tutorial uses these CSV files in the `data/` directory:
- `movies.csv`: title, released, tagline
- `people.csv`: name, born
- `genres.csv`: name
- `acted_in.csv`: actor, movie, role
- `directed.csv`: director, movie
- `movie_genres.csv`: movie, genre